In [ ]:
!pip install statsmodels==0.14.1




In [ ]:
import psutil
print(f"Memory used: {psutil.Process().memory_info().rss / 1024**2:.2f} MB")


In [ ]:
import os, glob
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.multivariate.manova import MANOVA
from statsmodels.multivariate.multivariate_ols import _multivariate_ols_fit
from patsy import dmatrix
import statsmodels.formula.api as smf
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import AgglomerativeClustering
import statsmodels.api as sm
from scipy.optimize import minimize




In [ ]:
current_dct=os.getcwd()
csv_files=glob.glob(f"{current_dct}/*.csv")
dfs=[]
for files in csv_files:
    df=pd.read_csv(files)
    df["house_hold"] = os.path.splitext(os.path.basename(files))[0].split("H")[1]
    dfs.append(df)
data = pd.concat(dfs, ignore_index=True)

In [ ]:
data['index'] = pd.to_datetime(data['index']) 
data["house_hold"] = data["house_hold"].astype("int")
data = data.drop(columns=["PUMPE_TOT","TEMPERATURE:TOTAL"])
data["day"]=data["index"].dt.dayofyear
data['weekday'] = data['index'].dt.day_name().astype("category")
data["is_weekday"]=data['index'].apply(lambda x: 0 if x.weekday() < 5 else 1)
data["time_of_day"] = data["index"].dt.time
data["season"] = data["day"].apply(lambda x: math.sin((x - (31+28+24))/365 * 2*math.pi))
#data['is_summer'] = (data['season'] >= 0).astype(int)
data["is_weekday"]=data["is_weekday"].astype("category")
#data['is_summer'] = data['is_summer'].astype("category")

data["hour"] = data["index"].dt.hour

In [ ]:
data

In [ ]:
data_hourly = (
    data.groupby(['house_hold', 'day', 'hour'])
      .agg({
          'HAUSHALT_TOT': 'mean',
          'weekday': 'first',    # keep weekday info per day
          'season': 'first',
          'is_weekday': 'first' ,# keep season info per day
      })
      .reset_index()
)

print(data_hourly)

In [ ]:
data_hourly.isnull().sum()

In [ ]:

for day_ in range(1, 366):
    tmp_0 = data[(data["day"] == day_) & (data["house_hold"] == 4)].reset_index()
    
    tmp = data_hourly[(data_hourly["day"] == day_) & (data_hourly["house_hold"] == 4)].reset_index()

    plt.figure(figsize=(10,5))
    plt.plot(tmp_0.index, tmp_0["HAUSHALT_TOT"], label='Raw data (15-min)')
    
    plt.plot(tmp["hour"], tmp["HAUSHALT_TOT"], label='Hourly aggregation', marker='o')
    
    plt.legend(loc="upper left")
    plt.ylabel("HAUSHALT_TOT")
    plt.title(f"Household 4, Day {day_}")
    plt.grid()
    plt.show()


In [ ]:

result_chunks = []

for household_id, group in data_hourly.groupby("house_hold"):
    temp = group.copy()

    # Rolling computations
    temp["v1"] = temp.groupby("day")["HAUSHALT_TOT"]\
                     .transform(lambda x: x.rolling(window=5, center=True, min_periods=1).mean())
    temp["v2"] = temp.groupby(["weekday", "hour"])["HAUSHALT_TOT"]\
                     .transform(lambda x: x.rolling(window=5, center=True, min_periods=1).mean())

    # Pivot per household
    pivot = temp.pivot_table(
        index=["house_hold", "day", "weekday", "season", "is_weekday"],
        columns='hour',
        values=['v1', 'v2']
    )

    pivot.columns = [f'{val}_hour_{hour}' for val, hour in pivot.columns]
    result_chunks.append(pivot.reset_index())

# Combine all chunks
df = pd.concat(result_chunks, ignore_index=True)
df["house_hold"] = df["house_hold"].astype("category")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Filter only v1 columns
v1_cols = [col for col in df.columns if col.startswith("v1_hour_")]

# Calculate missing percentage per household for v1 only
missing_summary_v1 = df.groupby("house_hold")[v1_cols].apply(
    lambda group: group.isnull().sum().sum() / group.size * 100
).reset_index(name="missing_percentage")

# Plot
plt.figure(figsize=(8, 5))
plt.bar(missing_summary_v1['house_hold'].astype(str), missing_summary_v1['missing_percentage'], color='salmon')
plt.xlabel('Household')
plt.ylabel('Missing Value Percentage (%)')
plt.title('Missing Value Percentage by Household')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:

data_hourly["v1"] = data_hourly.groupby(["house_hold", "day"])["HAUSHALT_TOT"]\
    .transform(lambda x: x.rolling(window=5, center=True, min_periods=1).mean())

data_hourly["v2"]=data_hourly.groupby(["house_hold", "weekday", "hour"])["HAUSHALT_TOT"]\
    .transform(lambda x: x.rolling(window=5, center=True, min_periods=1).mean())



In [ ]:
for day_ in range(1, 366): #if we want to plot for each day of the year
    tmp = data_hourly[(data_hourly["day"] == day_) & (data_hourly["house_hold"] == 4)].reset_index()
    #plt.plot(tmp.index, tmp["HAUSHALT_TOT"], label = 'Haushalt total')
    plt.plot(tmp["hour"], tmp["HAUSHALT_TOT"], label = 'raw_data')
    plt.plot(tmp["hour"], tmp["v1"], label = 'Withinday smoothing')
    plt.plot(tmp["hour"], tmp["v2"], label = 'Weekday smoothing')
    plt.legend(loc="upper left")
    plt.ylabel("Haushalt total")
    plt.title("Day: "+ str(day_))
    #plt.xticks(ticks=tmp.index, labels=tmp["hour"])
    plt.grid()
    plt.show()
    

In [ ]:
hour_cols_v1 = [f'v1_hour_{h}' for h in range (0,24)]
df[hour_cols_v1]=df[hour_cols_v1].interpolate(axis=1, method='linear', limit_direction='both')
hour_cols_v2 = [f'v2_hour_{h}' for h in range (0,24)]
df[hour_cols_v2]=df[hour_cols_v2].interpolate(axis=1, method='linear', limit_direction='both')

In [ ]:
np.random.seed(123)
household_3=np.random.choice(df["house_hold"].unique(),3,replace=False)
df_3=df[df["house_hold"].isin(household_3)].copy()
df_30=df[~df["house_hold"].isin(household_3)].copy()
for col in ["house_hold", "weekday", "is_weekday"]:
    df_30[col] = df_30[col].cat.remove_unused_categories()
    df_3[col] = df_3[col].cat.remove_unused_categories()


In [ ]:
df_3["house_hold"].unique()

## manova for v1

In [ ]:
dependent_vars_v1 = '+'.join([f'v1_hour_{h}' for h in range (0,24)])
formula1=f'{dependent_vars_v1} ~ -1+ house_hold*weekday*season'
formula2=f'{dependent_vars_v1} ~ -1+house_hold*is_weekday*season'
manova_v1 = MANOVA.from_formula(formula1, data=df_30)
manova_v12 = MANOVA.from_formula(formula2, data=df_30)
print(manova_v1.mv_test())
print(manova_v12.mv_test())

## manova for v2

In [ ]:
dependent_vars_v2 = '+'.join([f'v2_hour_{h}' for h in range (0,24)])
formula1=f'{dependent_vars_v2} ~ -1+ house_hold*weekday*season'
formula2=f'{dependent_vars_v2} ~ -1+house_hold*is_weekday*season'
manova_v2 = MANOVA.from_formula(formula1, data=df_30)
manova_v22 = MANOVA.from_formula(formula2, data=df_30)
print(manova_v2.mv_test())
print(manova_v22.mv_test())

In [ ]:
def extract_wilks_lambda(manova_result):
    wilks_dict = {}

    for effect_name, effect_result in manova_result.mv_test().results.items():
        stat_df = effect_result['stat']
        wilks_row = stat_df.loc["Wilks' lambda"]

        wilks_dict[effect_name] = {
            'Wilks Lambda': wilks_row['Value'],
            'Num DF': wilks_row['Num DF'],
            'F Value': wilks_row['F Value'],
            'Pr > F': f"{wilks_row['Pr > F']:.4f}"
        }

    return wilks_dict



In [ ]:
wilks1 = extract_wilks_lambda(manova_v1)
wilks2 = extract_wilks_lambda(manova_v12)

import pandas as pd
df_wilks1 = pd.DataFrame.from_dict(wilks1, orient='index')
df_wilks2 = pd.DataFrame.from_dict(wilks2, orient='index')

print("MANOVA 1 Wilks' Lambda:")
print(df_wilks1)

print("\nMANOVA 2 Wilks' Lambda:")
print(df_wilks2)


In [ ]:
wilks1 = extract_wilks_lambda(manova_v2)
wilks2 = extract_wilks_lambda(manova_v22)

import pandas as pd
df_wilks1 = pd.DataFrame.from_dict(wilks1, orient='index')
df_wilks2 = pd.DataFrame.from_dict(wilks2, orient='index')

print("MANOVA 1 Wilks' Lambda:")
print(df_wilks1)

print("\nMANOVA 2 Wilks' Lambda:")
print(df_wilks2)


In [ ]:
def compute_aic_mv(model,Y, X):

    n, R = Y.shape
    q = X.shape[1]

    # Residuals
    #residuals = Y - np.dot(X,model[0]) # (n x R)
    residuals = Y - (X @ model[0])

    # Residual covariance matrix estimate (ML estimate)
    Sigma = model[3] / model[1]

    # Check if Sigma_ml is positive definite
    sign, logdet = np.linalg.slogdet(Sigma)
    if sign <= 0:
        raise ValueError("Covariance matrix is not positive definite.")

    # Mahalanobis distance squared for each observation
    Sigma_inv = np.linalg.inv(Sigma)
    maha2 = np.sum(residuals @ Sigma_inv * residuals, axis=1)
    #maha2 = np.einsum('ij,jk,ik->i', residuals, Sigma_inv, residuals)


    # Log-likelihood
    ll = - (n * R / 2) * np.log(2 * np.pi) - (n / 2) * logdet - 0.5 * np.sum(maha2)

    # Number of parameters: q*R regression + R(R+1)/2 covariance
    k = q * R + (R * (R + 1)) / 2

    # AIC
    AIC = 2 * k - 2 * ll

    return AIC


    

In [ ]:
X_main_effect_7 =pd.get_dummies(df_30[["house_hold", "weekday", "season"]], drop_first=True).astype("float")
X_interaction_7 = dmatrix("house_hold * weekday * season", data=df_30, return_type="dataframe")
X_main_effect_2 = pd.get_dummies(df_30[["house_hold", "is_weekday", "season"]], drop_first=True).astype("float")
X_interaction_2 =dmatrix("house_hold * is_weekday * season", data=df_30, return_type="dataframe")
Y = df_30[hour_cols_v1].to_numpy()
model_1=_multivariate_ols_fit(Y,X_main_effect_7)
model_2=_multivariate_ols_fit(Y,X_interaction_7)
model_3=_multivariate_ols_fit(Y,X_main_effect_2)
model_4=_multivariate_ols_fit(Y,X_interaction_2)
#params_1, df_resid_1, inv_cov_1, sscpr_1 = model_1
print(f"X_main_effect_7 AIC:{compute_aic_mv(model_1,Y, X_main_effect_7)}")
print(f"X_interaction_7 AIC:{compute_aic_mv(model_2,Y, X_interaction_7)}")
print(f"X_main_effect_2 AIC:{compute_aic_mv(model_3,Y, X_main_effect_2)}")
print(f"X_interaction_2 AIC:{compute_aic_mv(model_4,Y, X_interaction_2)}")

In [ ]:
X_main_effect_7 =pd.get_dummies(df_30[["house_hold", "weekday", "season"]], drop_first=True).astype("float")
X_interaction_7 = dmatrix("house_hold * weekday * season", data=df_30, return_type="dataframe")
X_main_effect_2 = pd.get_dummies(df_30[["house_hold", "is_weekday", "season"]], drop_first=True).astype("float")
X_interaction_2 =dmatrix("house_hold * is_weekday * season", data=df_30, return_type="dataframe")
Y = df_30[hour_cols_v2].to_numpy()
model_1=_multivariate_ols_fit(Y,X_main_effect_7)
model_2=_multivariate_ols_fit(Y,X_interaction_7)
model_3=_multivariate_ols_fit(Y,X_main_effect_2)
model_4=_multivariate_ols_fit(Y,X_interaction_2)
#params_1, df_resid_1, inv_cov_1, sscpr_1 = model_1
print(f"X_main_effect_7 AIC:{compute_aic_mv(model_1,Y, X_main_effect_7)}")
print(f"X_interaction_7 AIC:{compute_aic_mv(model_2,Y, X_interaction_7)}")
print(f"X_main_effect_2 AIC:{compute_aic_mv(model_3,Y, X_main_effect_2)}")
print(f"X_interaction_2 AIC:{compute_aic_mv(model_4,Y, X_interaction_2)}")

In [ ]:
for house_id in df_30["house_hold"].unique():
    house=df_30[df_30["house_hold"]==house_id]
    manova_3 = MANOVA.from_formula(f'{dependent_vars_v1} ~ -1+ is_weekday*season', data=house)
    print(f"houseld_hold{house_id} :{manova_3.mv_test()}")
    
    

## seperate houshold _v1


In [ ]:
for house_id in df_30["house_hold"].unique():
    house=df_30[df_30["house_hold"]==house_id]
    X_main_effect_7_separate =pd.get_dummies(house[["weekday", "season"]], drop_first=True).astype("float")
    X_interaction_7_separate = dmatrix(" weekday * season", data=house, return_type="dataframe")
    X_main_effect_2_separate= pd.get_dummies(house[["is_weekday", "season"]], drop_first=True).astype("float")
    X_interaction_2_separate =dmatrix(" is_weekday * season", data=house, return_type="dataframe")
    hour_cols_v1 = [f'v1_hour_{h}' for h in range (0,24)]
    Y = house[hour_cols_v1].to_numpy()
    model_1_separat=_multivariate_ols_fit(Y,X_main_effect_7_separate)
    model_2_separat=_multivariate_ols_fit(Y,X_interaction_7_separate)
    model_3_separat=_multivariate_ols_fit(Y,X_main_effect_2_separate)
    model_4_separat=_multivariate_ols_fit(Y,X_interaction_2_separate)
    #params_1, df_resid_1, inv_cov_1, sscpr_1 = model_1
    print(f"{house_id}X_main_effect_7 AIC:{compute_aic_mv(model_1_separat,Y, X_main_effect_7_separate)}")
    print(f"{house_id}X_interaction_7 AIC:{compute_aic_mv(model_2_separat,Y, X_interaction_7_separate)}")
    print(f"{house_id}X_main_effect_2 AIC:{compute_aic_mv(model_3_separat,Y, X_main_effect_2_separate)}")
    print(f"{house_id}X_interaction_2 AIC:{compute_aic_mv(model_4_separat,Y, X_interaction_2_separate)}")

## separete household v2

In [ ]:
for house_id in df_30["house_hold"].unique():
    house=df_30[df_30["house_hold"]==house_id]
    manova_3 = MANOVA.from_formula(f'{dependent_vars_v2} ~ -1+ is_weekday*season', data=house)
    print(f"{house_id} :{manova_3.mv_test()}")
    
    

In [ ]:
for house_id in df_30["house_hold"].unique():
    house=df_30[df_30["house_hold"]==house_id]
    X_main_effect_7_separate =pd.get_dummies(house[["weekday", "season"]], drop_first=True).astype("float")
    X_interaction_7_separate = dmatrix(" weekday * season", data=house, return_type="dataframe")
    X_main_effect_2_separate= pd.get_dummies(house[["is_weekday", "season"]], drop_first=True).astype("float")
    X_interaction_2_separate =dmatrix(" is_weekday * season", data=house, return_type="dataframe")
    hour_cols_v2 = [f'v2_hour_{h}' for h in range (0,24)]
    Y = house[hour_cols_v2].to_numpy()
    model_1_separat=_multivariate_ols_fit(Y,X_main_effect_7_separate)
    model_2_separat=_multivariate_ols_fit(Y,X_interaction_7_separate)
    model_3_separat=_multivariate_ols_fit(Y,X_main_effect_2_separate)
    model_4_separat=_multivariate_ols_fit(Y,X_interaction_2_separate)
    #params_1, df_resid_1, inv_cov_1, sscpr_1 = model_1
    print(f"{house_id}X_main_effect_7 AIC:{compute_aic_mv(model_1_separat,Y, X_main_effect_7_separate)}")
    print(f"{house_id}X_interaction_7 AIC:{compute_aic_mv(model_2_separat,Y, X_interaction_7_separate)}")
    print(f"{house_id}X_main_effect_2 AIC:{compute_aic_mv(model_3_separat,Y, X_main_effect_2_separate)}")
    print(f"{house_id}X_interaction_2 AIC:{compute_aic_mv(model_4_separat,Y, X_interaction_2_separate)}")

In [ ]:
import pandas as pd
from patsy import dmatrix

# will hold one row per household
rows = []

for house_id in df_30["house_hold"].unique():
    house = df_30[df_30["house_hold"] == house_id]
    
    # design matrices
    X_main7     = pd.get_dummies(house[["weekday", "season"]], drop_first=True).astype(float)
    X_int7      = dmatrix("weekday * season",      data=house, return_type="dataframe")
    X_main2     = pd.get_dummies(house[["is_weekday", "season"]], drop_first=True).astype(float)
    X_int2      = dmatrix("is_weekday * season",   data=house, return_type="dataframe")
    
    # response
    hour_cols   = [f'v1_hour_{h}' for h in range(24)]
    Y           = house[hour_cols].to_numpy()
    
    # fit each
    m1 = _multivariate_ols_fit(Y, X_main7)
    m2 = _multivariate_ols_fit(Y, X_int7)
    m3 = _multivariate_ols_fit(Y, X_main2)
    m4 = _multivariate_ols_fit(Y, X_int2)
    
    # compute AICs
    aic1 = compute_aic_mv(m1, Y, X_main7)
    aic2 = compute_aic_mv(m2, Y, X_int7)
    aic3 = compute_aic_mv(m3, Y, X_main2)
    aic4 = compute_aic_mv(m4, Y, X_int2)
    
    # flag if model 4 (2-level interaction) is best
    best_aic = min(aic1, aic2, aic3, aic4)
    interaction_needed_2 = (aic4 == best_aic)
    
    rows.append({
        "house_hold":               house_id,
        "AIC_main7":                aic1,
        "AIC_interaction7":         aic2,
        "AIC_main2":                aic3,
        "AIC_interaction2":         aic4,
        "use_interaction2":         interaction_needed_2
    })

# assemble into a DataFrame
df_model_selection = pd.DataFrame(rows)

print(df_model_selection)


In [ ]:
import pandas as pd
from patsy import dmatrix

# will hold one row per household
rows = []

for house_id in df_30["house_hold"].unique():
    house = df_30[df_30["house_hold"] == house_id]
    
    # design matrices
    X_main7     = pd.get_dummies(house[["weekday", "season"]], drop_first=True).astype(float)
    X_int7      = dmatrix("weekday * season",      data=house, return_type="dataframe")
    X_main2     = pd.get_dummies(house[["is_weekday", "season"]], drop_first=True).astype(float)
    X_int2      = dmatrix("is_weekday * season",   data=house, return_type="dataframe")
    
    # response
    hour_cols   = [f'v2_hour_{h}' for h in range(24)]
    Y           = house[hour_cols].to_numpy()
    
    # fit each
    m1 = _multivariate_ols_fit(Y, X_main7)
    m2 = _multivariate_ols_fit(Y, X_int7)
    m3 = _multivariate_ols_fit(Y, X_main2)
    m4 = _multivariate_ols_fit(Y, X_int2)
    
    # compute AICs
    aic1 = compute_aic_mv(m1, Y, X_main7)
    aic2 = compute_aic_mv(m2, Y, X_int7)
    aic3 = compute_aic_mv(m3, Y, X_main2)
    aic4 = compute_aic_mv(m4, Y, X_int2)
    
    # flag if model 4 (2-level interaction) is best
    best_aic = min(aic1, aic2, aic3, aic4)
    interaction_needed_4 = (aic2 == best_aic)
    
    rows.append({
        "house_hold":               house_id,
        "AIC_main7":                aic1,
        "AIC_interaction7":         aic2,
        "AIC_main2":                aic3,
        "AIC_interaction2":         aic4,
        "use_interaction_7":         interaction_needed_4
    })

# assemble into a DataFrame
df_model_selection = pd.DataFrame(rows)

print(df_model_selection)


##silhutte_plot_function

In [ ]:
from sklearn.metrics import silhouette_score

def plot_silhouette_scores_hierarchical(param_dict, title="Silhouette Score vs Number of Clusters (Hierarchical)"):
    param_vectors = []

    # Flatten coefficient matrices
    for coef_matrix in param_dict.values():
        param_vectors.append(coef_matrix)

    # Stack and scale
    X = np.vstack(param_vectors)
    X_scaled = StandardScaler().fit_transform(X)

    silhouette_scores = []

    for k in range(2, 7):
        model = AgglomerativeClustering(n_clusters=k)
        labels = model.fit_predict(X_scaled)
        score = silhouette_score(X_scaled, labels)
        silhouette_scores.append(score)
        print(f"k = {k}, Silhouette Score = {score:.4f}")

    # Plotting
    plt.figure(figsize=(8, 4))
    plt.plot(range(2, 7), silhouette_scores, marker='o')
    plt.title(title)
    plt.xlabel("Number of Clusters (k)")
    plt.ylabel("Silhouette Score")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


#clustering_function

In [ ]:
def cluster_and_pca(param_dict, n_clusters=2, n_neighbors=3):
    house_ids = []
    param_vectors = []

    # Flatten coefficient matrices
    for house_id, coef_matrix in param_dict.items():
        flat = coef_matrix.flatten()
        param_vectors.append(flat)
        house_ids.append(house_id)

    # Feature matrix
    X = np.vstack(param_vectors)

    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Hierarchical clustering
    hierarchical = AgglomerativeClustering(n_clusters=n_clusters)
    labels = hierarchical.fit_predict(X_scaled)

    # Create DataFrame of results
    cluster_df = pd.DataFrame({
        "house_id": house_ids,
        "cluster": labels
    })

    # Train classifier (KNN)
    classifier = KNeighborsClassifier(n_neighbors=n_neighbors)
    classifier.fit(X_scaled, labels)

    # PCA for visualization
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)

    return {
        "cluster_df": cluster_df,
        "pca_coordinates": X_pca,
        "labels": labels,
        "scaler": scaler,
        "classifier": classifier,  # trained KNN
        "model": hierarchical
    }


In [ ]:
param_30 = {}

for house_id in df_30["house_hold"].unique():
    house = df_30[df_30["house_hold"] == house_id].copy()
    house["is_weekday"] = house["is_weekday"].astype("category")


    # Design matrix without intercept (remove intercept with -1)
    X_interaction_2_separate = dmatrix(
        "C(is_weekday) * season ", data=house, return_type="dataframe"
    )


    hour_cols_v1 = [f'v1_hour_{h}' for h in range (0,24)]
    Y = house[hour_cols_v1].to_numpy()

    try:
        model_3 = _multivariate_ols_fit(Y, X_interaction_2_separate)
        param_30[house_id] = model_3[0]


    except ValueError as e:
        print(f"Skipped household {house_id}: {e}")


In [ ]:
param_30

In [ ]:
plot_silhouette_scores_hierarchical(param_30)

In [ ]:
clusters_with_week_weekend=cluster_and_pca(param_30)

fig, ax = plt.subplots(1, 1, figsize=(4, 4))

X_pca = clusters_with_week_weekend["pca_coordinates"]
labels = clusters_with_week_weekend["labels"]

scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='Set1', s=50)

ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
ax.set_title('Households clustered with weekdays and weekends')
ax.legend(*scatter.legend_elements(), title="Clusters")

plt.tight_layout()
plt.show()


In [ ]:
clusters_with_week_weekend = cluster_and_pca(param_30)

fig, ax = plt.subplots(1, 1, figsize=(4, 4))

X_pca = clusters_with_week_weekend["pca_coordinates"]
labels = clusters_with_week_weekend["labels"]

scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='Set1', s=50)

ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
ax.set_title('Households clustered with full model')
ax.legend(*scatter.legend_elements(), title="Clusters")

# Add silhouette score annotation
ax.text(0.95, 0.05, "Silhouette Score = 0.69", transform=ax.transAxes,
        fontsize=10, ha='right', va='bottom', bbox=dict(facecolor='white', edgecolor='gray'))

plt.tight_layout()
plt.show()


In [ ]:
X=clusters_with_week_weekend["cluster_df"]
Y_0=X[X["cluster"] == 0]
Y_1=X[X["cluster"] == 1]
print(Y_0.shape)
print(Y_1.shape)


In [ ]:


param_week_0 = {}
param_week_1 = {}
weekday_predict_df={}
weekend_predict_df={}

for house_id in df_30["house_hold"].unique():
    house = df_30[df_30["house_hold"] == house_id]

    house_0 = house[house["is_weekday"] == 0]
    house_1 = house[house["is_weekday"] == 1]

    # Add intercept (constant term) to the season variable
    X_main_effect_2_separate_0 = sm.add_constant(house_0[["season"]].reset_index(drop=True))
    X_main_effect_2_separate_1 = sm.add_constant(house_1[["season"]].reset_index(drop=True))

    hour_cols_v1 = [f'v1_hour_{h}' for h in range(24)]
    Y_0 = house_0[hour_cols_v1].to_numpy()
    Y_1 = house_1[hour_cols_v1].to_numpy()
    
    model_3_0 = _multivariate_ols_fit(Y_0, X_main_effect_2_separate_0)
    model_3_1 = _multivariate_ols_fit(Y_1, X_main_effect_2_separate_1)
    
    param_week_0[house_id] = model_3_0[0]
    param_week_1[house_id] = model_3_1[0]
    weekday_predict_df[house_id] = X_main_effect_2_separate_0 @ model_3_0[0]
    weekend_predict_df[house_id] = X_main_effect_2_separate_1 @ model_3_1[0]


    pred_0 = pd.DataFrame(X_main_effect_2_separate_0 @ model_3_0[0])
    pred_0.columns = hour_cols_v1

    pred_0["house_hold"] = house_id
    pred_0["day"] = house_0["day"].values
    weekday_predict_df[house_id] = pred_0

    pred_1 = pd.DataFrame(X_main_effect_2_separate_1 @ model_3_1[0])
    pred_1.columns = hour_cols_v1

    pred_1["house_hold"] = house_id
    pred_1["day"] = house_1["day"].values
    weekend_predict_df[house_id] = pred_1
    





In [ ]:
weekend_predict_df = pd.concat(weekend_predict_df.values(), ignore_index=True)
weekday_predict_df = pd.concat(weekday_predict_df.values(), ignore_index=True)


In [ ]:
weekend_predict_df

In [ ]:

   

# # Example usage:
# plot_silhouette_scores(param_week_0, title="Silhouette Scores for Weekdays")
# # or
# plot_silhouette_scores(param_week_1, title="Silhouette Scores for Weekends")


In [ ]:
plot_silhouette_scores_hierarchical(param_week_0, title="Silhouette Scores for Weekdays")
plot_silhouette_scores_hierarchical(param_week_1, title="Silhouette Scores for Weekends")

In [ ]:

weekday= cluster_and_pca(param_week_0)
weekend=cluster_and_pca(param_week_1)
#weekend["cluster_df"]["cluster"] = weekend["cluster_df"]["cluster"].map({0: 1, 1: 0})


In [ ]:
weekend["cluster_df"][weekend["cluster_df"]["cluster"] == 0].shape


In [ ]:
weekday["cluster_df"][weekday["cluster_df"]["cluster"] == 0].shape

In [ ]:
# from scipy.spatial.distance import cdist
# import numpy as np

# # Step 1: Extract cluster centers
# weekday_centers = weekday["kmeans_model"].cluster_centers_
# weekend_centers = weekend["kmeans_model"].cluster_centers_

# # Step 2: Calculate pairwise distances
# distance_matrix = cdist(weekday_centers, weekend_centers)

# # Step 3: Determine optimal alignment (weekday → weekend cluster index)
# weekday_to_weekend = distance_matrix.argmin(axis=1)

# # Step 4: Invert the mapping to remap weekend labels to weekday cluster meaning
# weekend_remap = {weekend_idx: weekday_idx for weekday_idx, weekend_idx in enumerate(weekday_to_weekend)}

# # Step 5: Apply remapping to weekend labels
# weekend["labels"] = np.vectorize(weekend_remap.get)(weekend["labels"])


# fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# for ax, data, label in zip(axs, [weekday, weekend], ["Weekday", "Weekend"]):
#     X_pca = data["pca_coordinates"]
#     clusters = data["labels"]
    
#     scatter = ax.scatter(
#         X_pca[:, 0], 
#         X_pca[:, 1], 
#         c=clusters, 
#         cmap='Set1',  # Color scheme changed here
#         s=50
#     )
#     ax.set_xlabel("PCA Component 1")
#     ax.set_ylabel("PCA Component 2")
#     ax.set_title(f"Households clustered by {label}")
#     ax.legend(*scatter.legend_elements(), title="Clusters")

# plt.tight_layout()
# plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assume:
# - weekday["pca_coordinates"] contains PCA results (n_samples x 2)
# - weekday["labels"] contains cluster labels
# - same for weekend

fig, axs = plt.subplots(1, 2, figsize=(16, 6))

for ax, data, label in zip(axs, [weekday, weekend], ["Weekday", "Weekend"]):
    X_pca = data["pca_coordinates"]
    clusters = data["labels"]

    scatter = ax.scatter(
        X_pca[:, 0],
        X_pca[:, 1],
        c=clusters,
        cmap='Set1',   # Change color map if desired
        s=50
    )
    ax.set_xlabel("PCA Component 1")
    ax.set_ylabel("PCA Component 2")
    ax.set_title(f"Hierarchical Clustering: {label}")
    ax.legend(*scatter.legend_elements(), title="Cluster")

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assume:
# - weekday["pca_coordinates"] contains PCA results (n_samples x 2)
# - weekday["labels"] contains cluster labels
# - same for weekend

fig, axs = plt.subplots(1, 2, figsize=(16, 6))

silhouette_scores = {"Weekday": 0.69, "Weekend": 0.68}

for ax, data, label in zip(axs, [weekday, weekend], ["Weekday", "Weekend"]):
    X_pca = data["pca_coordinates"]
    clusters = data["labels"]

    scatter = ax.scatter(
        X_pca[:, 0],
        X_pca[:, 1],
        c=clusters,
        cmap='Set1',
        s=50
    )
    ax.set_xlabel("PCA Component 1")
    ax.set_ylabel("PCA Component 2")
    ax.set_title(f"Hierarchical Clustering: {label}")
    ax.legend(*scatter.legend_elements(), title="Cluster")

    # Add silhouette score to bottom-right corner
    score = silhouette_scores[label]
    ax.text(
        0.95, 0.05,
        f"Silhouette Score = {score:.2f}",
        transform=ax.transAxes,
        fontsize=10,
        ha='right',
        va='bottom',
        bbox=dict(facecolor='white', edgecolor='gray')
    )

plt.tight_layout()
plt.show()


In [ ]:
param_30_rest= {}

for house_id in df_3["house_hold"].unique():
    house = df_3[df_3["house_hold"] == house_id].copy()
    house["is_weekday"] = house["is_weekday"].astype("category")


    # Design matrix without intercept
    X_interaction_2_separate = dmatrix(
        "C(is_weekday) * season ", data=house, return_type="dataframe"
    )


    hour_cols_v1 = [f'v1_hour_{h}' for h in range (0,24)]
    Y = house[hour_cols_v1].to_numpy()

    try:
        model_3 = _multivariate_ols_fit(Y, X_interaction_2_separate)
        param_30_rest[house_id] = model_3[0]


    except ValueError as e:
        print(f"Skipped household {house_id}: {e}")


In [ ]:
param_week_0_rest={}
param_week_1_rest={}

for house_id in df_3["house_hold"].unique():
    house = df_3[df_3["house_hold"] == house_id]

    house_0 = house[house["is_weekday"] == 0]
    house_1 = house[house["is_weekday"] == 1]




    X_main_effect_2_separate_0 = sm.add_constant(house_0[["season"]])
    X_main_effect_2_separate_1 = sm.add_constant(house_1[["season"]])


    hour_cols_v1 = [f'v1_hour_{h}' for h in range (0,24)]
    Y_0 = house_0[hour_cols_v1].to_numpy()
    Y_1 = house_1[hour_cols_v1].to_numpy()
    
    model_3_0 = _multivariate_ols_fit(Y_0, X_main_effect_2_separate_0)
    model_3_1 = _multivariate_ols_fit(Y_1, X_main_effect_2_separate_1)
    
    param_week_0_rest[house_id]=model_3_0[0]
    param_week_1_rest[house_id]=model_3_1[0]


In [ ]:
param_30_rest


## cluster Assignment


In [ ]:
def assign_clusters_to_new(param_dict_new, scaler, classifier):
    
    house_ids_new = []
    param_vectors_new = []

    for house_id, coef_matrix in param_dict_new.items():
        flat = coef_matrix.flatten()
        param_vectors_new.append(flat)
        house_ids_new.append(house_id)

    X_new = np.vstack(param_vectors_new)
    X_new_scaled = scaler.transform(X_new)

    new_labels = classifier.predict(X_new_scaled)

    new_cluster_df = pd.DataFrame({"house_id": house_ids_new, "cluster": new_labels})
    return new_cluster_df


In [ ]:
assign_clusters_to_new(param_30_rest, clusters_with_week_weekend["scaler"], clusters_with_week_weekend["classifier"])

In [ ]:
assign_clusters_to_new(param_week_0_rest, weekday["scaler"], weekday["classifier"])


In [ ]:
assign_clusters_to_new(param_week_1_rest, weekend["scaler"], weekend["classifier"])

In [ ]:
# Step 1: Extract mapping Series from weekday and weekend cluster_df
house_to_cluster_weekday = weekday["cluster_df"].set_index("house_id")["cluster"]
house_to_cluster_weekend = weekend["cluster_df"].set_index("house_id")["cluster"]

# Step 2: Map clusters to prediction DataFrames
weekday_predict_df["cluster"] = weekday_predict_df["house_hold"].map(house_to_cluster_weekday)
weekend_predict_df["cluster"] = weekend_predict_df["house_hold"].map(house_to_cluster_weekend)

In [ ]:

import pandas as pd

# Function to process each prediction DataFrame
def process_predict_df(predict_df):
    # Step 1: Melt wide → long format
    long_df = predict_df.melt(
        id_vars=["house_hold", "day", "cluster"],
        value_vars=[f"v1_hour_{h}" for h in range(24)],
        var_name="hour",
        value_name="value"
    )

    # Step 2: Extract hour number as integer
    long_df["hour"] = long_df["hour"].str.extract(r"v1_hour_(\d+)").astype(int)

    # Step 3: Compute aggregated average value per cluster/day/hour
    group_avg = (
        long_df
        .groupby(["cluster", "day", "hour"])["value"]
        .mean()
        .reset_index()
        .rename(columns={"value": "cluster_day_hour_avg"})
    )

    # Step 4: Merge average back to long format
    long_df_with_avg = long_df.merge(group_avg, on=["cluster", "day", "hour"])

    # Step 5: Group by house_hold, day, hour — keep raw value and cluster mean
    household_hourly = (
        long_df_with_avg
        .groupby(["house_hold", "day", "hour"])
        .agg(
            raw_value=("value", "first"),
            cluster=("cluster", "first"),
            cluster_day_hour_avg=("cluster_day_hour_avg", "first")
        )
        .reset_index()
    )

    return household_hourly

# Apply function to both datasets
weekday_hourly = process_predict_df(weekday_predict_df)
weekend_hourly = process_predict_df(weekend_predict_df)

# # Combine the results (optional)
# combined_hourly = pd.concat([weekday_hourly, weekend_hourly], ignore_index=True)

# # View sample
# print(combined_hourly.head())


In [ ]:
def compute_cluster_profiles(df, day):
    df_day = df[df["day"] == day].copy()
    df_day["squared_error"] = (df_day["raw_value"] - df_day["cluster_day_hour_avg"]) ** 2

    # Std per cluster/hour
    std_df = (
        df_day.groupby(["cluster", "hour"])["squared_error"]
        .mean()
        .reset_index()
        .rename(columns={"squared_error": "std"})
    )
    std_df["std"] = np.sqrt(std_df["std"])

    # Mean per cluster/hour
    mean_df = (
        df_day.groupby(["cluster", "hour"])["cluster_day_hour_avg"]
        .first()
        .reset_index()
    )

    # Merge and compute upper/lower bounds
    profile_df = mean_df.merge(std_df, on=["cluster", "hour"])
    profile_df["upper"] = profile_df["cluster_day_hour_avg"] + 3 * profile_df["std"]
    profile_df["lower"] = profile_df["cluster_day_hour_avg"] - 3 * profile_df["std"]

    return profile_df

In [ ]:
import matplotlib.pyplot as plt
import math

def plot_cluster_profiles(profile_df, title_prefix="", day=None):
    clusters = sorted(profile_df["cluster"].unique())
    n_clusters = len(clusters)
    n_rows = 2
    n_cols = 2

    fig, axs = plt.subplots(n_rows, n_cols, figsize=(14, 8), sharex=True, sharey=True)
    axs = axs.flatten()  # Flatten to index like a list

    for i, cluster in enumerate(clusters):
        subset = profile_df[profile_df["cluster"] == cluster]
        ax = axs[i]

        ax.plot(subset["hour"], subset["cluster_day_hour_avg"], label="Mean Load", linewidth=2)
        ax.fill_between(subset["hour"], subset["lower"], subset["upper"], alpha=0.3, label="±3σ Range")
        ax.set_title(f"{title_prefix}Cluster {cluster}" + (f" (Day {day})" if day else ""))
        ax.set_xlabel("Hour")
        ax.set_ylabel("Load")
        ax.grid(True)
        ax.legend()

    # Hide unused subplots if any
    for j in range(i + 1, len(axs)):
        fig.delaxes(axs[j])

    plt.tight_layout()
    plt.show()


In [ ]:
weekday_hourly = process_predict_df(weekday_predict_df)
weekend_hourly = process_predict_df(weekend_predict_df)



# Generate and plot weekday
weekday_profile = compute_cluster_profiles(weekday_hourly, 1)
plot_cluster_profiles(weekday_profile, title_prefix="Weekday ", day=1)

# Generate and plot weekend
weekend_profile = compute_cluster_profiles(weekend_hourly, 5)
plot_cluster_profiles(weekend_profile, title_prefix="Weekend ", day=5)


In [ ]:
len(df_3[df_3["house_hold"]==8])

## kalman fulter

In [ ]:
class KalmanForecaster:
    def __init__(self, Theta_X=0.8, Theta_v=0.2, theta_z=1.0, Q_diag=[1, 1, 1], R_diag=[10, 10, 10], P0=1.0):
        self.Theta_X = Theta_X
        self.Theta_v = Theta_v
        self.theta_z = theta_z
        self.Q_diag = Q_diag
        self.R_diag = R_diag
        self.P0 = P0
        self.n_vars = 3  # For 3 households

        # Core matrices (defined at fit time)
        self.F = np.eye(self.n_vars) * self.Theta_X
        self.B = np.eye(self.n_vars) * self.Theta_v
        self.H = np.eye(self.n_vars) * self.theta_z
        self.Q = np.diag(self.Q_diag)
        self.R = np.diag(self.R_diag)

    def fit(self, X_train, V_train=None):
        X = X_train[0, :]
        P = np.diag([self.P0] * self.n_vars)
        filtered_states = []

        for t in range(X_train.shape[0]):
            z = X_train[t]
            v = V_train[t] if V_train is not None and not isinstance(V_train, int) else np.zeros(self.n_vars)

            # Predict
            X_pred = self.F @ X + self.B @ v
            P_pred = self.F @ P @ self.F.T * self.Q

            # Update
            S = self.H @ P_pred @ self.H.T + self.R
            K = P_pred @ self.H.T @ np.linalg.inv(S)
            X = X_pred + K @ (z - self.H @ X_pred)
            P = (np.eye(self.n_vars) - K @ self.H) @ P_pred

            filtered_states.append(X.copy())

        self.last_X = X  # for forecasting
        self.last_P = P

        return pd.DataFrame(filtered_states, columns=["Filtered_8", "Filtered_11", "Filtered_34"])

    def forecast(self, V_test=None, steps=None):
        assert hasattr(self, 'last_X'), "You must call .fit() before .forecast()"
        X = self.last_X.copy()
        P = self.last_P.copy()
        forecasts = []

        if isinstance(V_test, int) or V_test is None:
            V_test = np.zeros((steps, self.n_vars))

        for t in range(V_test.shape[0]):
            v = V_test[t]
            X = self.F @ X + self.B @ v
            forecasts.append(X.copy())

        return pd.DataFrame(forecasts, columns=["Forecast_8", "Forecast_11", "Forecast_34"])


In [ ]:

import numpy as np
from sklearn.metrics import mean_squared_error

def multivariate_kalman_loss_per_household_qr(
    params,
    X_train,
    V_train=None,
    n_folds=2
):
    """
    Cross-validation loss for a multivariate Kalman filter with
    household-specific Q and R. If V_train is None, uses zeros.

    Parameters:
        params (list): [
            Theta_X, Theta_v, theta_z,
            q_8, q_11, q_34,
            r_8, r_11, r_34
        ]
        X_train (ndarray): observed data (n_timesteps, 3)
        V_train (ndarray or None): load-profile input (n_timesteps, 3)
        n_folds (int): number of CV folds

    Returns:
        float: average RMSE across households and folds
    """
    # Unpack the 9 parameters
    Theta_X, Theta_v, theta_z = params[:3]
    Q_diag = params[3:6]
    R_diag = params[6:9]

    # If no exogenous input provided, use zeros
    if V_train is None:
        V_train = np.zeros_like(X_train)

    n = len(X_train)
    fold_size = n // n_folds
    rmse_scores = []

    for fold in range(n_folds):
        start = fold * fold_size
        end = (fold + 1) * fold_size if fold < n_folds - 1 else n

        # Validation slice
        X_val = X_train[start:end]
        V_val = V_train[start:end]

        # Sub-training slice (everything except [start:end])
        X_sub = np.concatenate((X_train[:start], X_train[end:]), axis=0)
        V_sub = np.concatenate((V_train[:start], V_train[end:]), axis=0)

        # Build & fit the Kalman filter
        kf = KalmanForecaster(
            Theta_X=Theta_X,
            Theta_v=Theta_v,
            theta_z=theta_z,
            Q_diag=Q_diag,
            R_diag=R_diag
        )
        kf.fit(X_sub, V_sub)

        # Forecast on the held-out block
        forecast_df = kf.forecast(V_val)

        # Compute per-household RMSE
        rmse_8  = mean_squared_error(X_val[:, 0], forecast_df["Forecast_8"], squared=False)
        rmse_11 = mean_squared_error(X_val[:, 1], forecast_df["Forecast_11"], squared=False)
        rmse_34 = mean_squared_error(X_val[:, 2], forecast_df["Forecast_34"], squared=False)

        rmse_scores.append((rmse_8 + rmse_11 + rmse_34) / 3)

    return np.mean(rmse_scores)


## weeked kalman v1


In [ ]:

cluster_0 = weekend_hourly[weekend_hourly["cluster"] == 0]

load_profile_matrix = (
    cluster_0.groupby(["day", "hour"])["cluster_day_hour_avg"]
    .first()
    .unstack()
    .sort_index()
)
load_profile_flat = load_profile_matrix.values.flatten()

available_days = load_profile_matrix.index.tolist()
df_filtered = df_3[
    df_3["day"].isin(available_days) &
    df_3["house_hold"].isin([8, 11, 34])
].sort_values(by=["house_hold", "day"]).reset_index(drop=True)

hour_cols = [f"v1_hour_{i}" for i in range(24)]
households = [8, 11, 34]
observed_matrix = np.vstack([
    df_filtered[df_filtered["house_hold"] == h][hour_cols].values.flatten()
    for h in households
]).T

V_matrix = np.repeat(load_profile_flat.reshape(-1, 1), 3, axis=1)

split_point = int(0.5 * observed_matrix.shape[0])
X_train_50 = observed_matrix[:split_point]
X_test_50 = observed_matrix[split_point:]
V_train_50 = V_matrix[:split_point]
V_test_50 = V_matrix[split_point:]


In [ ]:
X_train_50.shape

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
import random
np.random.seed(42)
random.seed(42)

# Number of random trials
n_trials = 200

best_loss = np.inf
best_params = None

for trial in range(n_trials):
    # Sample parameters uniformly from your desired ranges
    Theta_X  = np.random.uniform(0.1, 0.99)
    Theta_v  = np.random.uniform(0.01, 1.0)
    theta_z  = np.random.uniform(0.1,  2.0)
    q8  = np.random.uniform(0.01, 10.0)
    q11 = np.random.uniform(0.01, 10.0)
    q34 = np.random.uniform(0.01, 10.0)
    r8  = np.random.uniform(0.1,  50.0)
    r11 = np.random.uniform(0.1,  50.0)
    r34 = np.random.uniform(0.1,  50.0)

    params = [Theta_X, Theta_v, theta_z, q8, q11, q34, r8, r11, r34]
    loss = multivariate_kalman_loss_per_household_qr(params, X_train_50, V_train_50, n_folds=3)

    if loss < best_loss:
        best_loss = loss
        best_params = params

    print(f"Trial {trial+1}/{n_trials}: RMSE = {loss:.2f}")

print("\n🎯 Best parameters found:")
print(f" Θₓ  = {best_params[0]:.4f}")
print(f" Θᵥ  = {best_params[1]:.4f}")
print(f" θ_z = {best_params[2]:.4f}")
print(f" q₈  = {best_params[3]:.4f}, q₁₁ = {best_params[4]:.4f}, q₃₄ = {best_params[5]:.4f}")
print(f" r₈  = {best_params[6]:.4f}, r₁₁ = {best_params[7]:.4f}, r₃₄ = {best_params[8]:.4f}")
print(f"Best cross‐validated RMSE: {best_loss:.2f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

# === Fit Kalman Filter on Training ===
kf = KalmanForecaster(
    Theta_X= 0.3550,
    Theta_v=0.5949,
    theta_z= 0.1580,
    Q_diag=[0.3831,8.2278, 3.6083],
    R_diag=[6.4403, 26.1599, 38.5227]
)

filtered_df = kf.fit(X_train_50, V_train_50)
forecast_df = kf.forecast(V_test_50)

# === Compute RMSE ===
rmse_fit_8 = mean_squared_error(X_train_50[:, 0], filtered_df["Filtered_8"], squared=False)
rmse_fit_11 = mean_squared_error(X_train_50[:, 1], filtered_df["Filtered_11"], squared=False)
rmse_fit_34 = mean_squared_error(X_train_50[:, 2], filtered_df["Filtered_34"], squared=False)

rmse_fore_8 = mean_squared_error(X_test_50[:, 0], forecast_df["Forecast_8"], squared=False)
rmse_fore_11 = mean_squared_error(X_test_50[:, 1], forecast_df["Forecast_11"], squared=False)
rmse_fore_34 = mean_squared_error(X_test_50[:, 2], forecast_df["Forecast_34"], squared=False)

# === Plot: Fitted Values ===
plt.figure(figsize=(14, 8))
plt.subplot(3, 1, 1)
plt.plot(X_train_50[:, 0], label="True 8")
plt.plot(filtered_df["Filtered_8"], label=f"Fitted 8 (RMSE={rmse_fit_8:.2f})")
plt.legend()
plt.title("Fitted - Household 8")

plt.subplot(3, 1, 2)
plt.plot(X_train_50[:, 1], label="True 11")
plt.plot(filtered_df["Filtered_11"], label=f"Fitted 11 (RMSE={rmse_fit_11:.2f})")
plt.legend()
plt.title("Fitted - Household 11")

plt.subplot(3, 1, 3)
plt.plot(X_train_50[:, 2], label="True 34")
plt.plot(filtered_df["Filtered_34"], label=f"Fitted 34 (RMSE={rmse_fit_34:.2f})")
plt.legend()
plt.title("Fitted - Household 34")

plt.tight_layout()
plt.show()

# === Plot: Forecasted Values ===
plt.figure(figsize=(14, 8))
plt.subplot(3, 1, 1)
plt.plot(X_test_50[:, 0], label="True 8")
plt.plot(forecast_df["Forecast_8"], label=f"Forecast 8 (RMSE={rmse_fore_8:.2f})")
plt.legend()
plt.title("Forecast - Household 8")

plt.subplot(3, 1, 2)
plt.plot(X_test_50[:, 1], label="True 11")
plt.plot(forecast_df["Forecast_11"], label=f"Forecast 11 (RMSE={rmse_fore_11:.2f})")
plt.legend()
plt.title("Forecast - Household 11")

plt.subplot(3, 1, 3)
plt.plot(X_test_50[:, 2], label="True 34")
plt.plot(forecast_df["Forecast_34"], label=f"Forecast 34 (RMSE={rmse_fore_34:.2f})")
plt.legend()
plt.title("Forecast - Household 34")

plt.tight_layout()
plt.show()


## weekday Kalman v1

In [ ]:
cluster_0 = weekday_hourly[weekday_hourly["cluster"] == 0]

load_profile_matrix = (
    cluster_0.groupby(["day", "hour"])["cluster_day_hour_avg"]
    .first()
    .unstack()
    .sort_index()
)
load_profile_flat = load_profile_matrix.values.flatten()

available_days = load_profile_matrix.index.tolist()
df_filtered = df_3[
    df_3["day"].isin(available_days) &
    df_3["house_hold"].isin([8, 11, 34])
].sort_values(by=["house_hold", "day"]).reset_index(drop=True)

hour_cols = [f"v1_hour_{i}" for i in range(24)]
households = [8, 11, 34]
observed_matrix = np.vstack([
    df_filtered[df_filtered["house_hold"] == h][hour_cols].values.flatten()
    for h in households
]).T

V_matrix = np.repeat(load_profile_flat.reshape(-1, 1), 3, axis=1)

split_point_week = int(0.5 * observed_matrix.shape[0])
X_train_50_week  = observed_matrix[:split_point]
X_test_50_week = observed_matrix[split_point:]
V_train_50_week  = V_matrix[:split_point]
V_test_50_week = V_matrix[split_point:]


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
np.random.seed(43)
random.seed(43)
# Number of random trials
n_trials = 200

best_loss = np.inf
best_params = None

for trial in range(n_trials):
    # Sample parameters uniformly from your desired ranges
    Theta_X  = np.random.uniform(0.1, 0.99)
    Theta_v  = np.random.uniform(0.01, 1.0)
    theta_z  = np.random.uniform(0.1,  2.0)
    q8  = np.random.uniform(0.01, 10.0)
    q11 = np.random.uniform(0.01, 10.0)
    q34 = np.random.uniform(0.01, 10.0)
    r8  = np.random.uniform(0.1,  50.0)
    r11 = np.random.uniform(0.1,  50.0)
    r34 = np.random.uniform(0.1,  50.0)

    params = [Theta_X, Theta_v, theta_z, q8, q11, q34, r8, r11, r34]
    loss = multivariate_kalman_loss_per_household_qr(params, X_train_50_week, V_train_50_week, n_folds=3)

    if loss < best_loss:
        best_loss = loss
        best_params = params

    print(f"Trial {trial+1}/{n_trials}: RMSE = {loss:.2f}")

print("\n🎯 Best parameters found:")
print(f" Θₓ  = {best_params[0]:.4f}")
print(f" Θᵥ  = {best_params[1]:.4f}")
print(f" θ_z = {best_params[2]:.4f}")
print(f" q₈  = {best_params[3]:.4f}, q₁₁ = {best_params[4]:.4f}, q₃₄ = {best_params[5]:.4f}")
print(f" r₈  = {best_params[6]:.4f}, r₁₁ = {best_params[7]:.4f}, r₃₄ = {best_params[8]:.4f}")
print(f"Best cross‐validated RMSE: {best_loss:.2f}")


In [ ]:
kf = KalmanForecaster(
    Theta_X=0.4970,
    Theta_v=0.4308,
    theta_z= 0.6257,
    Q_diag=[9.4448,7.3696, 9.9191],
    R_diag=[25.9738, 35.0268,  21.8945]
)

filtered_df_week = kf.fit(X_train_50_week, V_train_50_week)
forecast_df_week = kf.forecast(V_test_50_week)

# === Compute RMSE ===
rmse_fit_8 = mean_squared_error(X_train_50_week[:, 0], filtered_df_week["Filtered_8"], squared=False)
rmse_fit_11 = mean_squared_error(X_train_50_week[:, 1], filtered_df_week["Filtered_11"], squared=False)
rmse_fit_34 = mean_squared_error(X_train_50_week[:, 2], filtered_df_week["Filtered_34"], squared=False)

rmse_fore_8 = mean_squared_error(X_test_50_week[:, 0], forecast_df_week["Forecast_8"], squared=False)
rmse_fore_11 = mean_squared_error(X_test_50_week[:, 1], forecast_df_week["Forecast_11"], squared=False)
rmse_fore_34 = mean_squared_error(X_test_50_week[:, 2], forecast_df_week["Forecast_34"], squared=False)

# === Plot: Fitted Values ===
plt.figure(figsize=(14, 8))
plt.subplot(3, 1, 1)
plt.plot(X_train_50_week[:, 0], label="True 8")
plt.plot(filtered_df_week["Filtered_8"], label=f"Fitted 8 (RMSE={rmse_fit_8:.2f})")
plt.legend()
plt.title("Fitted - Household 8")

plt.subplot(3, 1, 2)
plt.plot(X_train_50_week[:, 1], label="True 11")
plt.plot(filtered_df_week["Filtered_11"], label=f"Fitted 11 (RMSE={rmse_fit_11:.2f})")
plt.legend()
plt.title("Fitted - Household 11")

plt.subplot(3, 1, 3)
plt.plot(X_train_50_week[:, 2], label="True 34")
plt.plot(filtered_df_week["Filtered_34"], label=f"Fitted 34 (RMSE={rmse_fit_34:.2f})")
plt.legend()
plt.title("Fitted - Household 34")

plt.tight_layout()
plt.show()

# === Plot: Forecasted Values ===
plt.figure(figsize=(14, 8))
plt.subplot(3, 1, 1)
plt.plot(X_test_50_week[:, 0], label="True 8")
plt.plot(forecast_df_week["Forecast_8"], label=f"Forecast 8 (RMSE={rmse_fore_8:.2f})")
plt.legend()
plt.title("Forecast - Household 8")

plt.subplot(3, 1, 2)
plt.plot(X_test_50_week[:, 1], label="True 11")
plt.plot(forecast_df_week["Forecast_11"], label=f"Forecast 11 (RMSE={rmse_fore_11:.2f})")
plt.legend()
plt.title("Forecast - Household 11")

plt.subplot(3, 1, 3)
plt.plot(X_test_50_week[:, 2], label="True 34")
plt.plot(forecast_df_week["Forecast_34"], label=f"Forecast 34 (RMSE={rmse_fore_34:.2f})")
plt.legend()
plt.title("Forecast - Household 34")

plt.tight_layout()
plt.show()



## without load v1 kalman

In [ ]:
import numpy as np

# 1) Define the hourly columns and target households
hour_columns = [f"v1_hour_{i}" for i in range(24)]
households   = [8, 11, 34]

# 2) For each household, sort by day and flatten its 365×24 values into one long series
series_list = []
for hh in households:
    hh_df = df_3[df_3["house_hold"] == hh].sort_values("day")
    # ensure 365 days × 24 hours
    assert hh_df.shape[0] == 365, f"Expected 365 days for HH {hh}, got {hh_df.shape[0]}"
    flat = hh_df[hour_columns].to_numpy().reshape(-1)  # length = 365*24
    series_list.append(flat)

# 3) Stack into an observed‐data matrix of shape (8760, 3)
observed_matrix = np.vstack(series_list).T

split_point_without_load = int(0.5 * observed_matrix.shape[0])
X_train_50_without_load  = observed_matrix[:split_point]
X_test_50_without_load = observed_matrix[split_point:]


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Number of random trials
n_trials = 200

best_loss = np.inf
best_params = None

for trial in range(n_trials):
    # Sample parameters uniformly from your desired ranges
    Theta_X  = np.random.uniform(0.1, 0.99)
    Theta_v  = np.random.uniform(0.01, 1.0)
    theta_z  = np.random.uniform(0.1,  2.0)
    q8  = np.random.uniform(0.01, 10.0)
    q11 = np.random.uniform(0.01, 10.0)
    q34 = np.random.uniform(0.01, 10.0)
    r8  = np.random.uniform(0.1,  50.0)
    r11 = np.random.uniform(0.1,  50.0)
    r34 = np.random.uniform(0.1,  50.0)

    params = [Theta_X, Theta_v, theta_z, q8, q11, q34, r8, r11, r34]
    loss = multivariate_kalman_loss_per_household_qr(params, X_train_50_without_load, n_folds=3)

    if loss < best_loss:
        best_loss = loss
        best_params = params

    print(f"Trial {trial+1}/{n_trials}: RMSE = {loss:.2f}")

print("\n🎯 Best parameters found:")
print(f" Θₓ  = {best_params[0]:.4f}")
print(f" Θᵥ  = {best_params[1]:.4f}")
print(f" θ_z = {best_params[2]:.4f}")
print(f" q₈  = {best_params[3]:.4f}, q₁₁ = {best_params[4]:.4f}, q₃₄ = {best_params[5]:.4f}")
print(f" r₈  = {best_params[6]:.4f}, r₁₁ = {best_params[7]:.4f}, r₃₄ = {best_params[8]:.4f}")
print(f"Best cross‐validated RMSE: {best_loss:.2f}"  """
    Random search tuning for your ParticleFilter.

    Parameters:
        X_train (ndarray): observed input (T, D)
        V_train (ndarray or None): exogenous input (T, D)
        V_future (ndarray or None): future V for prediction (steps, D)
        y_true (ndarray): ground truth for future (steps, D) — used for RMSE
        ParticleClass (class): your ParticleFilter class
        n_trials (int): number of random trials
        num_particles (int): number of particles in the filter
        seed (int): random seed

    Returns:
        tuple: (best_loss, best_params)
    """)

In [ ]:
# === Fit Kalman Filter on Training ===
kf = KalmanForecaster(
    Theta_X= 0.9890,
    Theta_v=0.8846,
    theta_z= 0.8312,
    Q_diag=[2.5687,4.2573, 1.1747],
    R_diag=[32.4325, 6.2845, 13.1311]
)

filtered_df_without_load = kf.fit(X_train_50_without_load)
forecast_df_without_load = kf.forecast(steps=X_test_50_without_load.shape[0])

# === Compute RMSE ===
rmse_fit_8 = mean_squared_error(X_train_50_without_load[:, 0], filtered_df_without_load["Filtered_8"], squared=False)
rmse_fit_11 = mean_squared_error(X_train_50_without_load[:, 1], filtered_df_without_load["Filtered_11"], squared=False)
rmse_fit_34 = mean_squared_error(X_train_50_without_load[:, 2], filtered_df_without_load["Filtered_34"], squared=False)

rmse_fore_8 = mean_squared_error(X_test_50_without_load[:, 0], forecast_df_without_load["Forecast_8"], squared=False)
rmse_fore_11 = mean_squared_error(X_test_50_without_load[:, 1], forecast_df_without_load["Forecast_11"], squared=False)
rmse_fore_34 = mean_squared_error(X_test_50_without_load[:, 2], forecast_df_without_load["Forecast_34"], squared=False)

# === Plot: Fitted Values ===
plt.figure(figsize=(14, 8))
plt.subplot(3, 1, 1)
plt.plot(X_train_50_without_load[:, 0], label="True 8")
plt.plot(filtered_df_without_load["Filtered_8"], label=f"Fitted 8 (RMSE={rmse_fit_8:.2f})")
plt.legend()
plt.title("Fitted - Household 8")

plt.subplot(3, 1, 2)
plt.plot(X_train_50_without_load[:, 1], label="True 11")
plt.plot(filtered_df_without_load["Filtered_11"], label=f"Fitted 11 (RMSE={rmse_fit_11:.2f})")
plt.legend()
plt.title("Fitted - Household 11")

plt.subplot(3, 1, 3)
plt.plot(X_train_50_without_load[:, 2], label="True 34")
plt.plot(filtered_df_without_load["Filtered_34"], label=f"Fitted 34 (RMSE={rmse_fit_34:.2f})")
plt.legend()
plt.title("Fitted - Household 34")

plt.tight_layout()
plt.show()

# === Plot: Forecasted Values ===
plt.figure(figsize=(14, 8))
plt.subplot(3, 1, 1)
plt.plot(X_test_50_without_load[:, 0], label="True 8")
plt.plot(forecast_df_without_load["Forecast_8"], label=f"Forecast 8 (RMSE={rmse_fore_8:.2f})")
plt.legend()
plt.title("Forecast - Household 8")

plt.subplot(3, 1, 2)
plt.plot(X_test_50_without_load[:, 1], label="True 11")
plt.plot(forecast_df_without_load["Forecast_11"], label=f"Forecast 11 (RMSE={rmse_fore_11:.2f})")
plt.legend()
plt.title("Forecast - Household 11")

plt.subplot(3, 1, 3)
plt.plot(X_test_50_without_load[:, 2], label="True 34")
plt.plot(forecast_df_without_load["Forecast_34"], label=f"Forecast 34 (RMSE={rmse_fore_34:.2f})")
plt.legend()
plt.title("Forecast - Household 34")

plt.tight_layout()
plt.show()



## particle_filter

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class ParticleFilter:
    def __init__(self,
                 num_particles=500,
                 theta_x=0.8,
                 theta_v=0.2,
                 theta_z=1.0,
                 process_noise_std=5.0,
                 measurement_noise_std=10.0):
        """
        A particle filter with optional exogenous input V and a linear measurement model z = θ_z * x + noise.
        """
        self.M = num_particles
        self.theta_x = theta_x
        self.theta_v = theta_v
        self.theta_z = theta_z
        self.q_std = process_noise_std
        self.r_std = measurement_noise_std

        self.particles = None    # shape (M, D)
        self.weights = None      # shape (M,)
        self.filtered = None     # DataFrame of filtered means

    def fit(self, X_obs, V_matrix=None):
        """
        Run the filter over observed data.
        
        Parameters
        ----------
        X_obs : ndarray, shape (T, D)
            Observed time‑series.
        V_matrix : ndarray, shape (T, D), optional
            Exogenous input; if None, filter runs without it.
        
        Returns
        -------
        pd.DataFrame
            Columns = ['Hour', 'Observed_0', …, 'Observed_{D-1}', 'Filtered_0', …, 'Filtered_{D-1}']
        """
        T, D = X_obs.shape
        # 1) initialize particles around the scale of the first observations
        self.particles = np.random.randn(self.M, D) * np.std(X_obs, axis=0)
        self.weights = np.ones(self.M) / self.M

        means = np.zeros((T, D))
        
        for t in range(T):
            # 2) Propagation step
            noise = np.random.randn(self.M, D) * self.q_std
            if V_matrix is not None:
                self.particles = (
                    self.theta_x * self.particles
                    + self.theta_v * V_matrix[t]
                    + noise
                )
            else:
                self.particles = (
                    self.theta_x * self.particles
                    + noise
                )
            
            # 3) Weighting step using z_t = theta_z * x_t + measurement noise
            obs = X_obs[t]  # shape (D,)
            # predicted measurements for each particle
            predicted_z = self.theta_z * self.particles      # (M, D)
            diffs = obs[None, :] - predicted_z               # (M, D)
            exponent = -0.5 * np.sum((diffs / self.r_std)**2, axis=1)
            coeff = (1 / np.sqrt(2 * np.pi * self.r_std**2))**D
            likelihood = coeff * np.exp(exponent)
            self.weights *= likelihood
            self.weights += 1e-300     # avoid zeros
            self.weights /= np.sum(self.weights)

            # 4) Estimation step: weighted mean of the state‐particles
            means[t] = np.average(self.particles, axis=0, weights=self.weights)

            # 5) Resampling step (multinomial)
            idx = np.random.choice(self.M, size=self.M, p=self.weights)
            self.particles = self.particles[idx]
            self.weights.fill(1.0 / self.M)

        # Package results into a DataFrame
        df = pd.DataFrame(means,
                          columns=[f"Filtered_{i}" for i in range(D)])
        df.insert(0, "Hour", np.arange(T))
        for i in range(D):
            df[f"Observed_{i}"] = X_obs[:, i]
        self.filtered = df
        return df

    def predict(self, V_future=None, steps=1):
        """
        Forecast future states via pure propagation (no measurement update).
        
        Parameters
        ----------
        V_future : ndarray, shape (steps, D), optional
            Future exogenous input. If None, predict without it.
        steps : int
            Number of future steps to generate.
        
        Returns
        -------
        pd.DataFrame
            Columns = ['Step', 'Predicted_0', …, 'Predicted_{D-1}']
        """
        D = self.particles.shape[1]
        preds = np.zeros((steps, D))

        for t in range(steps):
            noise = np.random.randn(self.M, D) * self.q_std
            if V_future is not None:
                self.particles = (
                    self.theta_x * self.particles
                    + self.theta_v * V_future[t]
                    + noise
                )
            else:
                self.particles = (
                    self.theta_x * self.particles
                    + noise
                )
            preds[t] = np.mean(self.particles, axis=0)

        df = pd.DataFrame(preds,
                          columns=[f"Predicted_{i}" for i in range(D)])
        df.insert(0, "Step", np.arange(1, steps+1))
        return df


#

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

def combined_filter_loss(
    params,
    X_train,
    V_train=None,
    n_folds=2,
    method="kalman",  # or "particle"
    KalmanClass=None,  # class for KalmanFilter
    ParticleClass=None  # class for ParticleFilter
):
    """
    Compute CV loss for either Kalman or Particle Filter.

    Parameters:
        params (list): [theta_x, theta_v, theta_z, q_8, q_11, q_34, r_8, r_11, r_34]
        X_train (ndarray): shape (T, 3), observed data
        V_train (ndarray or None): shape (T, 3), exogenous inputs
        n_folds (int): number of folds
        method (str): "kalman" or "particle"
        KalmanClass (class): Kalman filter class (must implement fit & forecast)
        ParticleClass (class): Particle filter class (must implement fit & predict)

    Returns:
        float: average RMSE across households and folds
    """
    theta_x, theta_v, theta_z = params[:3]
    q_values = params[3:6]
    r_values = params[6:9]

    if V_train is None:
        V_train = np.zeros_like(X_train)

    T = len(X_train)
    fold_size = T // n_folds
    rmse_scores = []

    for fold in range(n_folds):
        start = fold * fold_size
        end = (fold + 1) * fold_size if fold < n_folds - 1 else T

        X_val = X_train[start:end]
        V_val = V_train[start:end]
        X_sub = np.concatenate((X_train[:start], X_train[end:]), axis=0)
        V_sub = np.concatenate((V_train[:start], V_train[end:]), axis=0)

        rmses = []

        for i, (q, r) in enumerate(zip(q_values, r_values)):
            if method == "kalman":
                kf = KalmanClass(
                    Theta_X=theta_x,
                    Theta_v=theta_v,
                    theta_z=theta_z,
                    Q_diag=[q],
                    R_diag=[r]
                )
                kf.fit(X_sub[:, [i]], V_sub[:, [i]])
                forecast_df = kf.forecast(V_val[:, [i]])
                pred = forecast_df[f"Forecast_{[8,11,34][i]}"]

            elif method == "particle":
                pf = ParticleClass(
                    num_particles=500,
                    theta_x=theta_x,
                    theta_v=theta_v,
                    theta_z=theta_z,
                    process_noise_std=q,
                    measurement_noise_std=r
                )
                pf.fit(X_sub[:, [i]], V_sub[:, [i]])
                forecast_df = pf.predict(V_future=V_val[:, [i]], steps=len(X_val))
                pred = forecast_df["Predicted_0"]

            else:
                raise ValueError("method must be 'kalman' or 'particle'")

            rmse = mean_squared_error(X_val[:, i], pred, squared=False)
            rmses.append(rmse)

        rmse_scores.append(np.mean(rmses))

    return np.mean(rmse_scores)


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
np.random.seed(42)

def tune_particle_filter_on_training(
    X_train,
    V_train=None,
    ParticleClass=None,
    n_trials=100,
    num_particles=500,
    seed=42
):
    """
    Tune Particle Filter parameters using training error only (filtered vs observed).

    Parameters:
        X_train (ndarray): shape (T, D), observed data
        V_train (ndarray or None): shape (T, D), exogenous input (optional)
        ParticleClass (class): your ParticleFilter class
        n_trials (int): number of random configurations to try
        num_particles (int): number of particles
        seed (int): random seed for reproducibility

    Returns:
        (best_loss, best_params)
    """
    np.random.seed(seed)

    best_loss = np.inf
    best_params = None

    for trial in range(n_trials):
        # --- Random parameter sampling ---
        theta_x = np.random.uniform(0.5, 0.99)
        theta_v = np.random.uniform(0.0, 1.0)
        theta_z = np.random.uniform(0.5, 2.0)
        q_std   = np.random.uniform(0.01, 10.0)
        r_std   = np.random.uniform(0.1, 50.0)

        # --- Initialize and fit filter ---
        pf = ParticleClass(
            num_particles=num_particles,
            theta_x=theta_x,
            theta_v=theta_v,
            theta_z=theta_z,
            process_noise_std=q_std,
            measurement_noise_std=r_std
        )

        pf.fit(X_train, V_matrix=V_train)
        filtered = pf.filtered[[col for col in pf.filtered.columns if col.startswith("Filtered_")]].values

        # --- Compute RMSE between filtered states and original observed data ---
        loss = mean_squared_error(X_train, filtered, squared=False)

        print(f"Trial {trial+1:03d}: RMSE = {loss:.4f} | Params = {[theta_x, theta_v, theta_z, q_std, r_std]}")

        if loss < best_loss:
            best_loss = loss
            best_params = [theta_x, theta_v, theta_z, q_std, r_std]
            print("✅ New best!")

    print("\n🏆 Best RMSE on training:", best_loss)
    print("🎯 Best parameters:", best_params)

    return best_loss, best_params


In [ ]:
best_loss, best_params = tune_particle_filter_on_training(
    X_train=X_train_50_week,
    V_train=V_train_50_week,      # or None
    ParticleClass=ParticleFilter,
    n_trials=100
)


In [ ]:



pf = ParticleFilter(
    num_particles=1000,
    theta_x= 0.9025,
    theta_v=0.09248,
    theta_z= 0.9501,              # measurement gain
    process_noise_std=5.49896,
    measurement_noise_std= 42.3258
)

# 2) Fit on historical data, with or without V:
filtered_df = pf.fit(X_train_50_week, V_matrix=V_train_50_week)

# 3) Forecast next 24 hours:
#    provide V_future for with‐V forecast, or omit for pure propagation
forecast_df = pf.predict(V_future=V_test_50_week, steps=V_test_50_week.shape[0])



In [ ]:


import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

households = [8, 11, 34]

# --- 1) Compute RMSEs --- 
rmse_fit = []
for i, h in enumerate(households):
    true_train = X_train_50_week[:, i]
    pred_train = filtered_df[f"Filtered_{i}"]
    rmse_fit.append(mean_squared_error(true_train, pred_train, squared=False))

rmse_fore = []
for i, h in enumerate(households):
    true_test = X_test_50_week[:, i]
    pred_test = forecast_df[f"Predicted_{i}"]
    rmse_fore.append(mean_squared_error(true_test, pred_test, squared=False))

# --- 2) Plot Fitted vs True (training) ---
plt.figure(figsize=(14, 8))
for idx, h in enumerate(households, start=1):
    plt.subplot(3, 1, idx)
    plt.plot(X_train_50_week[:, idx-1], label=f"True {h}")
    plt.plot(filtered_df[f"Filtered_{idx-1}"],
             label=f"Fitted {h} (RMSE={rmse_fit[idx-1]:.2f})")
    plt.legend()
    plt.title(f"Fitted - Household {h}")
plt.tight_layout()
plt.show()

# --- 3) Plot Forecast vs True (test) ---
plt.figure(figsize=(14, 8))
for idx, h in enumerate(households, start=1):
    plt.subplot(3, 1, idx)
    plt.plot(X_test_50_week[:, idx-1], label=f"True {h}")
    plt.plot(forecast_df[f"Predicted_{idx-1}"],
             label=f"Forecast {h} (RMSE={rmse_fore[idx-1]:.2f})")
    plt.legend()
    plt.title(f"Forecast - Household {h}")
plt.tight_layout()
plt.show()


In [ ]:
best_loss, best_params = tune_particle_filter_on_training(
    X_train=X_train_50,
    V_train=V_train_50,      # or None
    ParticleClass=ParticleFilter,
    n_trials=100
)


In [ ]:

pf = ParticleFilter(
    num_particles=1000,
    theta_x=0.9025,
    theta_v= 0.09248,
    theta_z= 0.95012,              # measurement gain
    process_noise_std=5.4989,
    measurement_noise_std= 42.3258
    )

# 2) Fit on historical data, with or without V:
filtered_df = pf.fit(X_train_50, V_matrix=V_train_50)

# 3) Forecast next 24 hours:
#    provide V_future for with‐V forecast, or omit for pure propagation
forecast_df = pf.predict(V_future=V_test_50, steps=V_test_50.shape[0])



In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

households = [8, 11, 34]

# --- 1) Compute RMSEs --- 
rmse_fit = []
for i, h in enumerate(households):
    true_train = X_train_50[:, i]
    pred_train = filtered_df[f"Filtered_{i}"]
    rmse_fit.append(mean_squared_error(true_train, pred_train, squared=False))

rmse_fore = []
for i, h in enumerate(households):
    true_test = X_test_50[:, i]
    pred_test = forecast_df[f"Predicted_{i}"]
    rmse_fore.append(mean_squared_error(true_test, pred_test, squared=False))

# --- 2) Plot Fitted vs True (training) ---
plt.figure(figsize=(14, 8))
for idx, h in enumerate(households, start=1):
    plt.subplot(3, 1, idx)
    plt.plot(X_train_50[:, idx-1], label=f"True {h}")
    plt.plot(filtered_df[f"Filtered_{idx-1}"],
             label=f"Fitted {h} (RMSE={rmse_fit[idx-1]:.2f})")
    plt.legend()
    plt.title(f"Fitted - Household {h}")
plt.tight_layout()
plt.show()

# --- 3) Plot Forecast vs True (test) ---
plt.figure(figsize=(14, 8))
for idx, h in enumerate(households, start=1):
    plt.subplot(3, 1, idx)
    plt.plot(X_test_50[:, idx-1], label=f"True {h}")
    plt.plot(forecast_df[f"Predicted_{idx-1}"],
             label=f"Forecast {h} (RMSE={rmse_fore[idx-1]:.2f})")
    plt.legend()
    plt.title(f"Forecast - Household {h}")
plt.tight_layout()
plt.show()


In [ ]:
best_loss, best_params = tune_particle_filter_on_training(
    X_train=X_train_50_without_load,      # or None
    ParticleClass=ParticleFilter,
    n_trials=100
)


In [ ]:

pf = ParticleFilter(
    num_particles=1000,
    theta_x=0.9841,
    theta_v=0,
    theta_z=1.3609,              # measurement gain
    process_noise_std=8.21,
    measurement_noise_std=30.1722
)

# 2) Fit on historical data, with or without V:
filtered_df = pf.fit(X_train_50_without_load)

# 3) Forecast next 24 hours:
#    provide V_future for with‐V forecast, or omit for pure propagation
forecast_df = pf.predict(steps=X_test_50_without_load.shape[0])



In [ ]:
# assume you’ve already done:
# filtered_df   = pf.fit(X_train_50_week, V_matrix=V_train_50_week)
# forecast_df   = pf.predict(V_future=V_test_50_week, steps=V_test_50_week.shape[0])
# and that the true series live in X_train_50_week (shape T×3) and X_test_50_week (shape H×3)

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

households = [8, 11, 34]

# --- 1) Compute RMSEs --- 
rmse_fit = []
for i, h in enumerate(households):
    true_train = X_train_50_without_load[:, i]
    pred_train = filtered_df[f"Filtered_{i}"]
    rmse_fit.append(mean_squared_error(true_train, pred_train, squared=False))

rmse_fore = []
for i, h in enumerate(households):
    true_test = X_test_50_without_load[:, i]
    pred_test = forecast_df[f"Predicted_{i}"]
    rmse_fore.append(mean_squared_error(true_test, pred_test, squared=False))

# --- 2) Plot Fitted vs True (training) ---
plt.figure(figsize=(14, 8))
for idx, h in enumerate(households, start=1):
    plt.subplot(3, 1, idx)
    plt.plot(X_train_50_without_load[:, idx-1], label=f"True {h}")
    plt.plot(filtered_df[f"Filtered_{idx-1}"],
             label=f"Fitted {h} (RMSE={rmse_fit[idx-1]:.2f})")
    plt.legend()
    plt.title(f"Fitted - Household {h}")
plt.tight_layout()
plt.show()

# --- 3) Plot Forecast vs True (test) ---
plt.figure(figsize=(14, 8))
for idx, h in enumerate(households, start=1):
    plt.subplot(3, 1, idx)
    plt.plot(X_test_50_without_load[:, idx-1], label=f"True {h}")
    plt.plot(forecast_df[f"Predicted_{idx-1}"],
             label=f"Forecast {h} (RMSE={rmse_fore[idx-1]:.2f})")
    plt.legend()
    plt.title(f"Forecast - Household {h}")
plt.tight_layout()
plt.show()

## or V2


In [ ]:
param_30 = {}

for house_id in df_30["house_hold"].unique():
    house = df_30[df_30["house_hold"] == house_id].copy()
    house["is_weekday"] = house["is_weekday"].astype("category")


    # Design matrix without intercept (remove intercept with -1)
    X_interaction_2_separate = dmatrix(
        "C(is_weekday) * season ", data=house, return_type="dataframe"
    )


    hour_cols_v2 = [f'v2_hour_{h}' for h in range (0,24)]
    Y = house[hour_cols_v2].to_numpy()

    try:
        model_3 = _multivariate_ols_fit(Y, X_interaction_2_separate)
        param_30[house_id] = model_3[0]


    except ValueError as e:
        print(f"Skipped household {house_id}: {e}")

In [ ]:
plot_silhouette_scores_hierarchical(param_30)

In [ ]:
clusters_with_week_weekend = cluster_and_pca(param_30)

fig, ax = plt.subplots(1, 1, figsize=(4, 4))

X_pca = clusters_with_week_weekend["pca_coordinates"]
labels = clusters_with_week_weekend["labels"]

scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='Set1', s=50)

ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
ax.set_title('Households clustered with weekdays and weekends')
ax.legend(*scatter.legend_elements(), title="Clusters")

# Add silhouette score annotation in bottom right corner
ax.text(
    0.98, 0.02, 
    "Silhouette Score = 0.62", 
    transform=ax.transAxes,
    ha='right', va='bottom',
    fontsize=10, color='black'
)

plt.tight_layout()
plt.show()


In [ ]:
param_week_0 = {}
param_week_1 = {}
weekday_predict_df={}
weekend_predict_df={}

for house_id in df_30["house_hold"].unique():
    house = df_30[df_30["house_hold"] == house_id]

    house_0 = house[house["is_weekday"] == 0]
    house_1 = house[house["is_weekday"] == 1]

    # Add intercept (constant term) to the season variable
    X_main_effect_2_separate_0 = sm.add_constant(house_0[["season"]].reset_index(drop=True))
    X_main_effect_2_separate_1 = sm.add_constant(house_1[["season"]].reset_index(drop=True))

    hour_cols_v2 = [f'v2_hour_{h}' for h in range(24)]
    Y_0 = house_0[hour_cols_v2].to_numpy()
    Y_1 = house_1[hour_cols_v2].to_numpy()
    
    model_3_0 = _multivariate_ols_fit(Y_0, X_main_effect_2_separate_0)
    model_3_1 = _multivariate_ols_fit(Y_1, X_main_effect_2_separate_1)
    
    param_week_0[house_id] = model_3_0[0]
    param_week_1[house_id] = model_3_1[0]
    weekday_predict_df[house_id] = X_main_effect_2_separate_0 @ model_3_0[0]
    weekend_predict_df[house_id] = X_main_effect_2_separate_1 @ model_3_1[0]


    pred_0 = pd.DataFrame(X_main_effect_2_separate_0 @ model_3_0[0])
    pred_0.columns = hour_cols_v2

    pred_0["house_hold"] = house_id
    pred_0["day"] = house_0["day"].values
    weekday_predict_df[house_id] = pred_0

    pred_1 = pd.DataFrame(X_main_effect_2_separate_1 @ model_3_1[0])
    pred_1.columns = hour_cols_v2

    pred_1["house_hold"] = house_id
    pred_1["day"] = house_1["day"].values
    weekend_predict_df[house_id] = pred_1
    


In [ ]:

weekday= cluster_and_pca(param_week_0)
weekend=cluster_and_pca(param_week_1)

In [ ]:


plot_silhouette_scores_hierarchical(param_week_0, title="Silhouette Scores for Weekdays")
plot_silhouette_scores_hierarchical(param_week_1, title="Silhouette Scores for Weekends")


In [ ]:
weekend["cluster_df"][weekend["cluster_df"]["cluster"] == 0].shape

In [ ]:
weekday["cluster_df"][weekday["cluster_df"]["cluster"] == 0].shape

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Mapping semantic clusters to visual clusters
# Assume these remappings based on your observation
# Weekday: 0 = Red (Group A), 1 = Grey (Group B)
# Weekend: 1 = Red (Group A), 0 = Grey (Group B)

color_map = {0: 'grey', 1: 'red'}           # Weekday labels
weekend_remap = {0: 1, 1: 0}                # Reverse mapping for weekend

fig, axs = plt.subplots(1, 2, figsize=(16, 6))

for ax, data, label, sil_score, remap in zip(
    axs,
    [weekday, weekend],
    ["Weekday", "Weekend"],
    [0.63, 0.62],
    [False, True]  # apply remap only to weekend
):
    X_pca = data["pca_coordinates"]
    clusters = data["labels"]

    # Remap if necessary (for weekend)
    if remap:
        clusters = [weekend_remap[int(c)] for c in clusters]

    # Plot points with semantic cluster colors
    for i in range(len(X_pca)):
        color = color_map[int(clusters[i])]
        ax.scatter(X_pca[i, 0], X_pca[i, 1], color=color, s=50)

    ax.set_xlabel("PCA Component 1")
    ax.set_ylabel("PCA Component 2")
    ax.set_title(f"Hierarchical Clustering: {label}")

    # Silhouette score
    ax.text(
        0.98, 0.02,
        f"Silhouette Score = {sil_score:.2f}",
        transform=ax.transAxes,
        ha='right', va='bottom',
        fontsize=10
    )

    # Semantic legend (consistent across both)
    custom_legend = [
        Line2D([0], [0], marker='o', color='w', label='Cluster 0',
               markerfacecolor='red', markersize=8),
        Line2D([0], [0], marker='o', color='w', label='Cluster 1',
               markerfacecolor='grey', markersize=8)
    ]
    ax.legend(handles=custom_legend, title="Cluster")

plt.tight_layout()
plt.show()


In [ ]:
param_30_rest= {}

for house_id in df_3["house_hold"].unique():
    house = df_3[df_3["house_hold"] == house_id].copy()
    house["is_weekday"] = house["is_weekday"].astype("category")


    # Design matrix without intercept
    X_interaction_2_separate = dmatrix(
        "C(is_weekday) * season ", data=house, return_type="dataframe"
    )


    hour_cols_v2 = [f'v2_hour_{h}' for h in range (0,24)]
    Y = house[hour_cols_v2].to_numpy()

    try:
        model_3 = _multivariate_ols_fit(Y, X_interaction_2_separate)
        param_30_rest[house_id] = model_3[0]


    except ValueError as e:
        print(f"Skipped household {house_id}: {e}")

In [ ]:
param_week_0_rest={}
param_week_1_rest={}

for house_id in df_3["house_hold"].unique():
    house = df_3[df_3["house_hold"] == house_id]

    house_0 = house[house["is_weekday"] == 0]
    house_1 = house[house["is_weekday"] == 1]




    X_main_effect_2_separate_0 = sm.add_constant(house_0[["season"]])
    X_main_effect_2_separate_1 = sm.add_constant(house_1[["season"]])


    hour_cols_v2 = [f'v2_hour_{h}' for h in range (0,24)]
    Y_0 = house_0[hour_cols_v2].to_numpy()
    Y_1 = house_1[hour_cols_v2].to_numpy()
    
    model_3_0 = _multivariate_ols_fit(Y_0, X_main_effect_2_separate_0)
    model_3_1 = _multivariate_ols_fit(Y_1, X_main_effect_2_separate_1)
    
    param_week_0_rest[house_id]=model_3_0[0]
    param_week_1_rest[house_id]=model_3_1[0]

In [ ]:
assign_clusters_to_new(param_30_rest, clusters_with_week_weekend["scaler"], clusters_with_week_weekend["classifier"])

In [ ]:
assign_clusters_to_new(param_week_0_rest, weekday["scaler"], weekday["classifier"])

In [ ]:
assign_clusters_to_new(param_week_1_rest, weekend["scaler"], weekend["classifier"])

In [ ]:
weekend_predict_df = pd.concat(weekend_predict_df.values(), ignore_index=True)
weekday_predict_df = pd.concat(weekday_predict_df.values(), ignore_index=True)


In [ ]:
house_to_cluster_weekday = weekday["cluster_df"].set_index("house_id")["cluster"]
house_to_cluster_weekend = weekend["cluster_df"].set_index("house_id")["cluster"]

# Step 2: Map clusters to prediction DataFrames
weekday_predict_df["cluster"] = weekday_predict_df["house_hold"].map(house_to_cluster_weekday)
weekend_predict_df["cluster"] = weekend_predict_df["house_hold"].map(house_to_cluster_weekend)

In [ ]:
weekend_predict_df

In [ ]:

def process_predict_df(predict_df):
    # Step 1: Melt wide → long format
    long_df = predict_df.melt(
        id_vars=["house_hold", "day", "cluster"],
        value_vars=[f"v2_hour_{h}" for h in range(24)],
        var_name="hour",
        value_name="value"
    )

    # Step 2: Extract hour number as integer
    long_df["hour"] = long_df["hour"].str.extract(r"v2_hour_(\d+)").astype(int)

    # Step 3: Compute aggregated average value per cluster/day/hour
    group_avg = (
        long_df
        .groupby(["cluster", "day", "hour"])["value"]
        .mean()
        .reset_index()
        .rename(columns={"value": "cluster_day_hour_avg"})
    )

    # Step 4: Merge average back to long format
    long_df_with_avg = long_df.merge(group_avg, on=["cluster", "day", "hour"])

    # Step 5: Group by house_hold, day, hour — keep raw value and cluster mean
    household_hourly = (
        long_df_with_avg
        .groupby(["house_hold", "day", "hour"])
        .agg(
            raw_value=("value", "first"),
            cluster=("cluster", "first"),
            cluster_day_hour_avg=("cluster_day_hour_avg", "first")
        )
        .reset_index()
    )

    return household_hourly

# Apply function to both datasets
weekday_hourly = process_predict_df(weekday_predict_df)
weekend_hourly = process_predict_df(weekend_predict_df)

# # Combine the results (optional)
# combined_hourly = pd.concat([weekday_hourly, weekend_hourly], ignore_index=True)

# # View sample
# print(combined_hourly.head())


In [ ]:
weekday_hourly = process_predict_df(weekday_predict_df)
weekend_hourly = process_predict_df(weekend_predict_df)



weekday_profile = compute_cluster_profiles(weekday_hourly, 1)
plot_cluster_profiles(weekday_profile, title_prefix="Weekday ", day=1)


weekend_profile = compute_cluster_profiles(weekend_hourly, 5)
plot_cluster_profiles(weekend_profile, title_prefix="Weekend ", day=5)

## weekday kalman V2

In [ ]:
cluster_1 = weekday_hourly[weekday_hourly["cluster"] == 1]

load_profile_matrix = (
    cluster_0.groupby(["day", "hour"])["cluster_day_hour_avg"]
    .first()
    .unstack()
    .sort_index()
)
load_profile_flat = load_profile_matrix.values.flatten()

available_days = load_profile_matrix.index.tolist()
df_filtered = df_3[
    df_3["day"].isin(available_days) &
    df_3["house_hold"].isin([8, 11, 34])
].sort_values(by=["house_hold", "day"]).reset_index(drop=True)

hour_cols = [f"v2_hour_{i}" for i in range(24)]
households = [8, 11, 34]
observed_matrix = np.vstack([
    df_filtered[df_filtered["house_hold"] == h][hour_cols].values.flatten()
    for h in households
]).T

V_matrix = np.repeat(load_profile_flat.reshape(-1, 1), 3, axis=1)

split_point_week = int(0.5 * observed_matrix.shape[0])
X_train_50_week  = observed_matrix[:split_point]
X_test_50_week = observed_matrix[split_point:]
V_train_50_week  = V_matrix[:split_point]
V_test_50_week = V_matrix[split_point:]


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
np.random.seed(42)
random.seed(42)

n_trials = 200

best_loss = np.inf
best_params = None

for trial in range(n_trials):
    # Sample parameters uniformly from your desired ranges
    Theta_X  = np.random.uniform(0.1, 0.99)
    Theta_v  = np.random.uniform(0.01, 1.0)
    theta_z  = np.random.uniform(0.1,  2.0)
    q8  = np.random.uniform(0.01, 10.0)
    q11 = np.random.uniform(0.01, 10.0)
    q34 = np.random.uniform(0.01, 10.0)
    r8  = np.random.uniform(0.1,  50.0)
    r11 = np.random.uniform(0.1,  50.0)
    r34 = np.random.uniform(0.1,  50.0)

    params = [Theta_X, Theta_v, theta_z, q8, q11, q34, r8, r11, r34]
    loss = multivariate_kalman_loss_per_household_qr(params, X_train_50_week, V_train_50_week, n_folds=3)

    if loss < best_loss:
        best_loss = loss
        best_params = params

    print(f"Trial {trial+1}/{n_trials}: RMSE = {loss:.2f}")

print("\n🎯 Best parameters found:")
print(f" Θₓ  = {best_params[0]:.4f}")
print(f" Θᵥ  = {best_params[1]:.4f}")
print(f" θ_z = {best_params[2]:.4f}")
print(f" q₈  = {best_params[3]:.4f}, q₁₁ = {best_params[4]:.4f}, q₃₄ = {best_params[5]:.4f}")
print(f" r₈  = {best_params[6]:.4f}, r₁₁ = {best_params[7]:.4f}, r₃₄ = {best_params[8]:.4f}")
print(f"Best cross‐validated RMSE: {best_loss:.2f}")


In [ ]:
kf = KalmanForecaster(
    Theta_X=0.4145,
    Theta_v=0.5878,
    theta_z= 0.2477,
    Q_diag=[9.7442,9.8622, 6.9846],
    R_diag=[26.8512, 15.5454, 40.7084]
)

filtered_df_week = kf.fit(X_train_50_week, V_train_50_week)
forecast_df_week = kf.forecast(V_test_50_week)

# === Compute RMSE ===
rmse_fit_8 = mean_squared_error(X_train_50_week[:, 0], filtered_df_week["Filtered_8"], squared=False)
rmse_fit_11 = mean_squared_error(X_train_50_week[:, 1], filtered_df_week["Filtered_11"], squared=False)
rmse_fit_34 = mean_squared_error(X_train_50_week[:, 2], filtered_df_week["Filtered_34"], squared=False)

rmse_fore_8 = mean_squared_error(X_test_50_week[:, 0], forecast_df_week["Forecast_8"], squared=False)
rmse_fore_11 = mean_squared_error(X_test_50_week[:, 1], forecast_df_week["Forecast_11"], squared=False)
rmse_fore_34 = mean_squared_error(X_test_50_week[:, 2], forecast_df_week["Forecast_34"], squared=False)

# === Plot: Fitted Values ===
plt.figure(figsize=(14, 8))
plt.subplot(3, 1, 1)
plt.plot(X_train_50_week[:, 0], label="True 8")
plt.plot(filtered_df_week["Filtered_8"], label=f"Fitted 8 (RMSE={rmse_fit_8:.2f})")
plt.legend()
plt.title("Fitted - Household 8")

plt.subplot(3, 1, 2)
plt.plot(X_train_50_week[:, 1], label="True 11")
plt.plot(filtered_df_week["Filtered_11"], label=f"Fitted 11 (RMSE={rmse_fit_11:.2f})")
plt.legend()
plt.title("Fitted - Household 11")

plt.subplot(3, 1, 3)
plt.plot(X_train_50_week[:, 2], label="True 34")
plt.plot(filtered_df_week["Filtered_34"], label=f"Fitted 34 (RMSE={rmse_fit_34:.2f})")
plt.legend()
plt.title("Fitted - Household 34")

plt.tight_layout()
plt.show()

# === Plot: Forecasted Values ===
plt.figure(figsize=(14, 8))
plt.subplot(3, 1, 1)
plt.plot(X_test_50_week[:, 0], label="True 8")
plt.plot(forecast_df_week["Forecast_8"], label=f"Forecast 8 (RMSE={rmse_fore_8:.2f})")
plt.legend()
plt.title("Forecast - Household 8")

plt.subplot(3, 1, 2)
plt.plot(X_test_50_week[:, 1], label="True 11")
plt.plot(forecast_df_week["Forecast_11"], label=f"Forecast 11 (RMSE={rmse_fore_11:.2f})")
plt.legend()
plt.title("Forecast - Household 11")

plt.subplot(3, 1, 3)
plt.plot(X_test_50_week[:, 2], label="True 34")
plt.plot(forecast_df_week["Forecast_34"], label=f"Forecast 34 (RMSE={rmse_fore_34:.2f})")
plt.legend()
plt.title("Forecast - Household 34")

plt.tight_layout()
plt.show()

## weekend kalman v2

In [ ]:
# Extract cluster 0 and cluster 1 profiles
cluster_0 = weekend_hourly[weekend_hourly["cluster"] == 0]
cluster_1 = weekend_hourly[weekend_hourly["cluster"] == 1]

# Construct load profile matrices
profile_0 = (
    cluster_0.groupby(["day", "hour"])["cluster_day_hour_avg"]
    .first()
    .unstack()
    .sort_index()
)

profile_1 = (
    cluster_1.groupby(["day", "hour"])["cluster_day_hour_avg"]
    .first()
    .unstack()
    .sort_index()
)

# Flatten each profile
flat_0 = profile_0.values.flatten()
flat_1 = profile_1.values.flatten()

# Combine into V matrix (household 8 & 34 use cluster 0, household 11 uses cluster 1)
V_matrix = np.stack([
    flat_0,       # Household 8 (cluster 0)
    flat_1,       # Household 11 (cluster 1)
    flat_0        # Household 34 (cluster 0)
], axis=1)

# Filter observed data to match available days
available_days = profile_0.index.tolist()  # assumes profile_0 and profile_1 use the same days
df_filtered = df_3[
    df_3["day"].isin(available_days) &
    df_3["house_hold"].isin([8, 11, 34])
].sort_values(by=["house_hold", "day"]).reset_index(drop=True)

# Create observed matrix
hour_cols = [f"v2_hour_{i}" for i in range(24)]
households = [8, 11, 34]
observed_matrix = np.vstack([
    df_filtered[df_filtered["house_hold"] == h][hour_cols].values.flatten()
    for h in households
]).T

# Split into training and test sets
split_point = int(0.5 * observed_matrix.shape[0])
X_train_50 = observed_matrix[:split_point]
X_test_50 = observed_matrix[split_point:]
V_train_50 = V_matrix[:split_point]
V_test_50 = V_matrix[split_point:]



In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
import random
np.random.seed(42)
random.seed(42)

# Number of random trials
n_trials = 200

best_loss = np.inf
best_params = None

for trial in range(n_trials):
    # Sample parameters uniformly from your desired ranges
    Theta_X  = np.random.uniform(0.1, 0.99)
    Theta_v  = np.random.uniform(0.01, 1.0)
    theta_z  = np.random.uniform(0.1,  2.0)
    q8  = np.random.uniform(0.01, 10.0)
    q11 = np.random.uniform(0.01, 10.0)
    q34 = np.random.uniform(0.01, 10.0)
    r8  = np.random.uniform(0.1,  50.0)
    r11 = np.random.uniform(0.1,  50.0)
    r34 = np.random.uniform(0.1,  50.0)

    params = [Theta_X, Theta_v, theta_z, q8, q11, q34, r8, r11, r34]
    loss = multivariate_kalman_loss_per_household_qr(params, X_train_50, V_train_50, n_folds=3)

    if loss < best_loss:
        best_loss = loss
        best_params = params

    print(f"Trial {trial+1}/{n_trials}: RMSE = {loss:.2f}")

print("\n🎯 Best parameters found:")
print(f" Θₓ  = {best_params[0]:.4f}")
print(f" Θᵥ  = {best_params[1]:.4f}")
print(f" θ_z = {best_params[2]:.4f}")
print(f" q₈  = {best_params[3]:.4f}, q₁₁ = {best_params[4]:.4f}, q₃₄ = {best_params[5]:.4f}")
print(f" r₈  = {best_params[6]:.4f}, r₁₁ = {best_params[7]:.4f}, r₃₄ = {best_params[8]:.4f}")
print(f"Best cross‐validated RMSE: {best_loss:.2f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

# === Fit Kalman Filter on Training ===
kf = KalmanForecaster(
    Theta_X= 0.3222,
    Theta_v=0.4945,
    theta_z= 0.5203,
    Q_diag=[7.0276,9.4412, 0.4039],
    R_diag=[35.3082, 46.2699,9.1107]
)

filtered_df = kf.fit(X_train_50, V_train_50)
forecast_df = kf.forecast(V_test_50)

# === Compute RMSE ===
rmse_fit_8 = mean_squared_error(X_train_50[:, 0], filtered_df["Filtered_8"], squared=False)
rmse_fit_11 = mean_squared_error(X_train_50[:, 1], filtered_df["Filtered_11"], squared=False)
rmse_fit_34 = mean_squared_error(X_train_50[:, 2], filtered_df["Filtered_34"], squared=False)

rmse_fore_8 = mean_squared_error(X_test_50[:, 0], forecast_df["Forecast_8"], squared=False)
rmse_fore_11 = mean_squared_error(X_test_50[:, 1], forecast_df["Forecast_11"], squared=False)
rmse_fore_34 = mean_squared_error(X_test_50[:, 2], forecast_df["Forecast_34"], squared=False)

# === Plot: Fitted Values ===
plt.figure(figsize=(14, 8))
plt.subplot(3, 1, 1)
plt.plot(X_train_50[:, 0], label="True 8")
plt.plot(filtered_df["Filtered_8"], label=f"Fitted 8 (RMSE={rmse_fit_8:.2f})")
plt.legend()
plt.title("Fitted - Household 8")

plt.subplot(3, 1, 2)
plt.plot(X_train_50[:, 1], label="True 11")
plt.plot(filtered_df["Filtered_11"], label=f"Fitted 11 (RMSE={rmse_fit_11:.2f})")
plt.legend()
plt.title("Fitted - Household 11")

plt.subplot(3, 1, 3)
plt.plot(X_train_50[:, 2], label="True 34")
plt.plot(filtered_df["Filtered_34"], label=f"Fitted 34 (RMSE={rmse_fit_34:.2f})")
plt.legend()
plt.title("Fitted - Household 34")

plt.tight_layout()
plt.show()

# === Plot: Forecasted Values ===
plt.figure(figsize=(14, 8))
plt.subplot(3, 1, 1)
plt.plot(X_test_50[:, 0], label="True 8")
plt.plot(forecast_df["Forecast_8"], label=f"Forecast 8 (RMSE={rmse_fore_8:.2f})")
plt.legend()
plt.title("Forecast - Household 8")

plt.subplot(3, 1, 2)
plt.plot(X_test_50[:, 1], label="True 11")
plt.plot(forecast_df["Forecast_11"], label=f"Forecast 11 (RMSE={rmse_fore_11:.2f})")
plt.legend()
plt.title("Forecast - Household 11")

plt.subplot(3, 1, 3)
plt.plot(X_test_50[:, 2], label="True 34")
plt.plot(forecast_df["Forecast_34"], label=f"Forecast 34 (RMSE={rmse_fore_34:.2f})")
plt.legend()
plt.title("Forecast - Household 34")

plt.tight_layout()
plt.show()

## v2 kalman without load

In [ ]:
# 1) Define the hourly columns and target households
hour_columns = [f"v2_hour_{i}" for i in range(24)]
households   = [8, 11, 34]

# 2) For each household, sort by day and flatten its 365×24 values into one long series
series_list = []
for hh in households:
    hh_df = df_3[df_3["house_hold"] == hh].sort_values("day")
    # ensure 365 days × 24 hours
    assert hh_df.shape[0] == 365, f"Expected 365 days for HH {hh}, got {hh_df.shape[0]}"
    flat = hh_df[hour_columns].to_numpy().reshape(-1)  # length = 365*24
    series_list.append(flat)

# 3) Stack into an observed‐data matrix of shape (8760, 3)
observed_matrix = np.vstack(series_list).T

split_point_without_load = int(0.5 * observed_matrix.shape[0])
X_train_50_without_load  = observed_matrix[:split_point]
X_test_50_without_load = observed_matrix[split_point:]

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Number of random trials
n_trials = 200

best_loss = np.inf
best_params = None

for trial in range(n_trials):
    # Sample parameters uniformly from your desired ranges
    Theta_X  = np.random.uniform(0.1, 0.99)
    Theta_v  = np.random.uniform(0.01, 1.0)
    theta_z  = np.random.uniform(0.1,  2.0)
    q8  = np.random.uniform(0.01, 10.0)
    q11 = np.random.uniform(0.01, 10.0)
    q34 = np.random.uniform(0.01, 10.0)
    r8  = np.random.uniform(0.1,  50.0)
    r11 = np.random.uniform(0.1,  50.0)
    r34 = np.random.uniform(0.1,  50.0)

    params = [Theta_X, Theta_v, theta_z, q8, q11, q34, r8, r11, r34]
    loss = multivariate_kalman_loss_per_household_qr(params, X_train_50_without_load, n_folds=3)

    if loss < best_loss:
        best_loss = loss
        best_params = params

    print(f"Trial {trial+1}/{n_trials}: RMSE = {loss:.2f}")

print("\n🎯 Best parameters found:")
print(f" Θₓ  = {best_params[0]:.4f}")
print(f" Θᵥ  = {best_params[1]:.4f}")
print(f" θ_z = {best_params[2]:.4f}")
print(f" q₈  = {best_params[3]:.4f}, q₁₁ = {best_params[4]:.4f}, q₃₄ = {best_params[5]:.4f}")
print(f" r₈  = {best_params[6]:.4f}, r₁₁ = {best_params[7]:.4f}, r₃₄ = {best_params[8]:.4f}")
print(f"Best cross‐validated RMSE: {best_loss:.2f}" )


In [ ]:
kf = KalmanForecaster(
    Theta_X= 0.9890,
    Theta_v=0.3334,
    theta_z= 1.5215,
    Q_diag=[8.0686,8.5817, 9.9763],
    R_diag=[12.1507,2.1135, 20.6185]
)

filtered_df_without_load = kf.fit(X_train_50_without_load)
forecast_df_without_load = kf.forecast(steps=X_test_50_without_load.shape[0])

# === Compute RMSE ===
rmse_fit_8 = mean_squared_error(X_train_50_without_load[:, 0], filtered_df_without_load["Filtered_8"], squared=False)
rmse_fit_11 = mean_squared_error(X_train_50_without_load[:, 1], filtered_df_without_load["Filtered_11"], squared=False)
rmse_fit_34 = mean_squared_error(X_train_50_without_load[:, 2], filtered_df_without_load["Filtered_34"], squared=False)

rmse_fore_8 = mean_squared_error(X_test_50_without_load[:, 0], forecast_df_without_load["Forecast_8"], squared=False)
rmse_fore_11 = mean_squared_error(X_test_50_without_load[:, 1], forecast_df_without_load["Forecast_11"], squared=False)
rmse_fore_34 = mean_squared_error(X_test_50_without_load[:, 2], forecast_df_without_load["Forecast_34"], squared=False)

# === Plot: Fitted Values ===
plt.figure(figsize=(14, 8))
plt.subplot(3, 1, 1)
plt.plot(X_train_50_without_load[:, 0], label="True 8")
plt.plot(filtered_df_without_load["Filtered_8"], label=f"Fitted 8 (RMSE={rmse_fit_8:.2f})")
plt.legend()
plt.title("Fitted - Household 8")

plt.subplot(3, 1, 2)
plt.plot(X_train_50_without_load[:, 1], label="True 11")
plt.plot(filtered_df_without_load["Filtered_11"], label=f"Fitted 11 (RMSE={rmse_fit_11:.2f})")
plt.legend()
plt.title("Fitted - Household 11")

plt.subplot(3, 1, 3)
plt.plot(X_train_50_without_load[:, 2], label="True 34")
plt.plot(filtered_df_without_load["Filtered_34"], label=f"Fitted 34 (RMSE={rmse_fit_34:.2f})")
plt.legend()
plt.title("Fitted - Household 34")

plt.tight_layout()
plt.show()

# === Plot: Forecasted Values ===
plt.figure(figsize=(14, 8))
plt.subplot(3, 1, 1)
plt.plot(X_test_50_without_load[:, 0], label="True 8")
plt.plot(forecast_df_without_load["Forecast_8"], label=f"Forecast 8 (RMSE={rmse_fore_8:.2f})")
plt.legend()
plt.title("Forecast - Household 8")

plt.subplot(3, 1, 2)
plt.plot(X_test_50_without_load[:, 1], label="True 11")
plt.plot(forecast_df_without_load["Forecast_11"], label=f"Forecast 11 (RMSE={rmse_fore_11:.2f})")
plt.legend()
plt.title("Forecast - Household 11")

plt.subplot(3, 1, 3)
plt.plot(X_test_50_without_load[:, 2], label="True 34")
plt.plot(forecast_df_without_load["Forecast_34"], label=f"Forecast 34 (RMSE={rmse_fore_34:.2f})")
plt.legend()
plt.title("Forecast - Household 34")

plt.tight_layout()
plt.show()

## particle filter v2 

In [ ]:
cluster_1 = weekday_hourly[weekday_hourly["cluster"] == 1]

load_profile_matrix = (
    cluster_0.groupby(["day", "hour"])["cluster_day_hour_avg"]
    .first()
    .unstack()
    .sort_index()
)
load_profile_flat = load_profile_matrix.values.flatten()

available_days = load_profile_matrix.index.tolist()
df_filtered = df_3[
    df_3["day"].isin(available_days) &
    df_3["house_hold"].isin([8, 11, 34])
].sort_values(by=["house_hold", "day"]).reset_index(drop=True)

hour_cols = [f"v2_hour_{i}" for i in range(24)]
households = [8, 11, 34]
observed_matrix = np.vstack([
    df_filtered[df_filtered["house_hold"] == h][hour_cols].values.flatten()
    for h in households
]).T

V_matrix = np.repeat(load_profile_flat.reshape(-1, 1), 3, axis=1)

split_point_week = int(0.5 * observed_matrix.shape[0])
X_train_50_week  = observed_matrix[:split_point]
X_test_50_week = observed_matrix[split_point:]
V_train_50_week  = V_matrix[:split_point]
V_test_50_week = V_matrix[split_point:]

In [ ]:
best_loss, best_params = tune_particle_filter_on_training(
    X_train=X_train_50_week,
    V_train=V_train_50_week,      # or None
    ParticleClass=ParticleFilter,
    n_trials=100
)


In [ ]:
pf = ParticleFilter(
    num_particles=1000,
    theta_x=0.8,
    theta_v=0.2,
    theta_z=1.81,              # measurement gain
    process_noise_std=6.88,
    measurement_noise_std= 37.882
)

# 2) Fit on historical data, with or without V:
filtered_df = pf.fit(X_train_50_week, V_matrix=V_train_50_week)

# 3) Forecast next 24 hours:
#    provide V_future for with‐V forecast, or omit for pure propagation
forecast_df = pf.predict(V_future=V_test_50_week, steps=V_test_50_week.shape[0])


In [ ]:
households = [8, 11, 34]

# --- 1) Compute RMSEs --- 
rmse_fit = []
for i, h in enumerate(households):
    true_train = X_train_50_week[:, i]
    pred_train = filtered_df[f"Filtered_{i}"]
    rmse_fit.append(mean_squared_error(true_train, pred_train, squared=False))

rmse_fore = []
for i, h in enumerate(households):
    true_test = X_test_50_week[:, i]
    pred_test = forecast_df[f"Predicted_{i}"]
    rmse_fore.append(mean_squared_error(true_test, pred_test, squared=False))

# --- 2) Plot Fitted vs True (training) ---
plt.figure(figsize=(14, 8))
for idx, h in enumerate(households, start=1):
    plt.subplot(3, 1, idx)
    plt.plot(X_train_50_week[:, idx-1], label=f"True {h}")
    plt.plot(filtered_df[f"Filtered_{idx-1}"],
             label=f"Fitted {h} (RMSE={rmse_fit[idx-1]:.2f})")
    plt.legend()
    plt.title(f"Fitted - Household {h}")
plt.tight_layout()
plt.show()

# --- 3) Plot Forecast vs True (test) ---
plt.figure(figsize=(14, 8))
for idx, h in enumerate(households, start=1):
    plt.subplot(3, 1, idx)
    plt.plot(X_test_50_week[:, idx-1], label=f"True {h}")
    plt.plot(forecast_df[f"Predicted_{idx-1}"],
             label=f"Forecast {h} (RMSE={rmse_fore[idx-1]:.2f})")
    plt.legend()
    plt.title(f"Forecast - Household {h}")
plt.tight_layout()
plt.show()


## particle filter v2 weekend

In [ ]:
cluster_0 = weekend_hourly[weekend_hourly["cluster"] == 0]
cluster_1 = weekend_hourly[weekend_hourly["cluster"] == 1]

# Construct load profile matrices
profile_0 = (
    cluster_0.groupby(["day", "hour"])["cluster_day_hour_avg"]
    .first()
    .unstack()
    .sort_index()
)

profile_1 = (
    cluster_1.groupby(["day", "hour"])["cluster_day_hour_avg"]
    .first()
    .unstack()
    .sort_index()
)

# Flatten each profile
flat_0 = profile_0.values.flatten()
flat_1 = profile_1.values.flatten()

# Combine into V matrix (household 8 & 34 use cluster 0, household 11 uses cluster 1)
V_matrix = np.stack([
    flat_0,       # Household 8 (cluster 0)
    flat_1,       # Household 11 (cluster 1)
    flat_0        # Household 34 (cluster 0)
], axis=1)

# Filter observed data to match available days
available_days = profile_0.index.tolist()  # assumes profile_0 and profile_1 use the same days
df_filtered = df_3[
    df_3["day"].isin(available_days) &
    df_3["house_hold"].isin([8, 11, 34])
].sort_values(by=["house_hold", "day"]).reset_index(drop=True)

# Create observed matrix
hour_cols = [f"v2_hour_{i}" for i in range(24)]
households = [8, 11, 34]
observed_matrix = np.vstack([
    df_filtered[df_filtered["house_hold"] == h][hour_cols].values.flatten()
    for h in households
]).T

# Split into training and test sets
split_point = int(0.5 * observed_matrix.shape[0])
X_train_50 = observed_matrix[:split_point]
X_test_50 = observed_matrix[split_point:]
V_train_50 = V_matrix[:split_point]
V_test_50 = V_matrix[split_point:]



In [ ]:
best_loss, best_params = tune_particle_filter_on_training(
    X_train=X_train_50,
    V_train=V_train_50,      # or None
    ParticleClass=ParticleFilter,
    n_trials=100
)

In [ ]:
pf = ParticleFilter(
    num_particles=1000,
    theta_x=0.9025,
    theta_v= 0.09248,
    theta_z=0.9501,              # measurement gain
    process_noise_std=5.49896,
    measurement_noise_std= 42.325
)

# 2) Fit on historical data, with or without V:
filtered_df = pf.fit(X_train_50, V_matrix=V_train_50)

# 3) Forecast next 24 hours:
#    provide V_future for with‐V forecast, or omit for pure propagation
forecast_df = pf.predict(V_future=V_test_50, steps=V_test_50.shape[0])

In [ ]:
households = [8, 11, 34]

# --- 1) Compute RMSEs --- 
rmse_fit = []
for i, h in enumerate(households):
    true_train = X_train_50[:, i]
    pred_train = filtered_df[f"Filtered_{i}"]
    rmse_fit.append(mean_squared_error(true_train, pred_train, squared=False))

rmse_fore = []
for i, h in enumerate(households):
    true_test = X_test_50[:, i]
    pred_test = forecast_df[f"Predicted_{i}"]
    rmse_fore.append(mean_squared_error(true_test, pred_test, squared=False))

# --- 2) Plot Fitted vs True (training) ---
plt.figure(figsize=(14, 8))
for idx, h in enumerate(households, start=1):
    plt.subplot(3, 1, idx)
    plt.plot(X_train_50[:, idx-1], label=f"True {h}")
    plt.plot(filtered_df[f"Filtered_{idx-1}"],
             label=f"Fitted {h} (RMSE={rmse_fit[idx-1]:.2f})")
    plt.legend()
    plt.title(f"Fitted - Household {h}")
plt.tight_layout()
plt.show()

# --- 3) Plot Forecast vs True (test) ---
plt.figure(figsize=(14, 8))
for idx, h in enumerate(households, start=1):
    plt.subplot(3, 1, idx)
    plt.plot(X_test_50[:, idx-1], label=f"True {h}")
    plt.plot(forecast_df[f"Predicted_{idx-1}"],
             label=f"Forecast {h} (RMSE={rmse_fore[idx-1]:.2f})")
    plt.legend()
    plt.title(f"Forecast - Household {h}")
plt.tight_layout()
plt.show()


In [ ]:
# 1) Define the hourly columns and target households
hour_columns = [f"v2_hour_{i}" for i in range(24)]
households   = [8, 11, 34]

# 2) For each household, sort by day and flatten its 365×24 values into one long series
series_list = []
for hh in households:
    hh_df = df_3[df_3["house_hold"] == hh].sort_values("day")
    # ensure 365 days × 24 hours
    assert hh_df.shape[0] == 365, f"Expected 365 days for HH {hh}, got {hh_df.shape[0]}"
    flat = hh_df[hour_columns].to_numpy().reshape(-1)  # length = 365*24
    series_list.append(flat)

# 3) Stack into an observed‐data matrix of shape (8760, 3)
observed_matrix = np.vstack(series_list).T

split_point_without_load = int(0.5 * observed_matrix.shape[0])
X_train_50_without_load  = observed_matrix[:split_point]
X_test_50_without_load = observed_matrix[split_point:]

In [ ]:
best_loss, best_params = tune_particle_filter_on_training(
    X_train=X_train_50_without_load,   
    ParticleClass=ParticleFilter,
    n_trials=100
)


In [ ]:
pf = ParticleFilter(
    num_particles=1000,
    theta_x=0.98,
    theta_v=0.2,
    theta_z=1.360,              # measurement gain
    process_noise_std=8.21,
    measurement_noise_std=30.17
)

# 2) Fit on historical data, with or without V:
filtered_df = pf.fit(X_train_50_without_load)

# 3) Forecast next 24 hours:
#    provide V_future for with‐V forecast, or omit for pure propagation
forecast_df = pf.predict(steps=X_test_50_without_load.shape[0])



In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

households = [8, 11, 34]

# --- 1) Compute RMSEs --- 
rmse_fit = []
for i, h in enumerate(households):
    true_train = X_train_50_without_load[:, i]
    pred_train = filtered_df[f"Filtered_{i}"]
    rmse_fit.append(mean_squared_error(true_train, pred_train, squared=False))

rmse_fore = []
for i, h in enumerate(households):
    true_test = X_test_50_without_load[:, i]
    pred_test = forecast_df[f"Predicted_{i}"]
    rmse_fore.append(mean_squared_error(true_test, pred_test, squared=False))

# --- 2) Plot Fitted vs True (training) ---
plt.figure(figsize=(14, 8))
for idx, h in enumerate(households, start=1):
    plt.subplot(3, 1, idx)
    plt.plot(X_train_50_without_load[:, idx-1], label=f"True {h}")
    plt.plot(filtered_df[f"Filtered_{idx-1}"],
             label=f"Fitted {h} (RMSE={rmse_fit[idx-1]:.2f})")
    plt.legend()
    plt.title(f"Fitted - Household {h}")
plt.tight_layout()
plt.show()

# --- 3) Plot Forecast vs True (test) ---
plt.figure(figsize=(14, 8))
for idx, h in enumerate(households, start=1):
    plt.subplot(3, 1, idx)
    plt.plot(X_test_50_without_load[:, idx-1], label=f"True {h}")
    plt.plot(forecast_df[f"Predicted_{idx-1}"],
             label=f"Forecast {h} (RMSE={rmse_fore[idx-1]:.2f})")
    plt.legend()
    plt.title(f"Forecast - Household {h}")
plt.tight_layout()
plt.show()